# Forecasting Consensus Expectations: Nonfarm Payrolls (NFP)

# Point and Directional Forecasts

**Imports**

In [ ]:

import os
import warnings
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st

from tqdm.auto import tqdm
from scipy import stats, special
from scipy.optimize import brentq, minimize
from scipy.stats import t as student_t, norm, binomtest, jarque_bera
from sklearn.mixture import GaussianMixture
from collections import defaultdict
from itertools import product
from arch.univariate import ConstantMean, GARCH, StudentsT
from arch.univariate.base import ConvergenceWarning
from IPython.display import display, Markdown
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import acf

warnings.filterwarnings("ignore")

In [122]:
OUT_DIR = "../out"         
DF_FILE       = "nfp_df.parquet"
DF_FULL_FILE  = "nfp_df_full.parquet"

df       = pd.read_parquet(os.path.join(OUT_DIR, DF_FILE),      engine="pyarrow")
df_full  = pd.read_parquet(os.path.join(OUT_DIR, DF_FULL_FILE), engine="pyarrow")

print("df shape     :", df.shape)
print("df_full shape:", df_full.shape)

df shape     : (16995, 10)
df_full shape: (19324, 10)


In [123]:
df.head()

,release_date,period,actual,median_forecast,economist,firm,forecast,asof,surprise,error
0,2003-06-06,2003-05-31,-17.000,-30.000,Alessandro Truppia,Aletti Gestielle Sgr Spa,-15.000,2003-06-04,13.000,2.000
1,2003-06-06,2003-05-31,-17.000,-30.000,Alison Lynn Reaser,Point Loma Nazarene University,-15.000,2003-05-30,13.000,2.000
2,2003-06-06,2003-05-31,-17.000,-30.000,Anthony Chan,JPMorgan Chase Bank,-70.000,2003-05-30,13.000,-53.000
3,2003-06-06,2003-05-31,-17.000,-30.000,Aurelio Maccario,UniCredit Spa,-9.000,2003-05-30,13.000,8.000
4,2003-06-06,2003-05-31,-17.000,-30.000,Avery Shenfeld,Canadian Imperial Bank of Commerce,-20.000,2003-05-30,13.000,-3.000


# 1 Point Forecast Ensembles

We deploy the following methods for point and directional forecasts: 
1. Static inverse-error
2. EWMA
3. Soft BMA with student-t likelihoods
4. Multiplicative weights update
5. Robust majority-vote ensemble

In [124]:
PANELS       = {"COVID": df.copy(), "Full": df_full.copy()}

# hyperparameters
CONT_WINDOWS = [3, 6, 12]                 # look‑back windows
METHODS      = ["inverse_mse", "inverse_mae", "equal_weight"]       # inverse error ensemble and EWMA
RIDGE        = 1e-6        # for numerical stability, not regularization (across all methods)
NU_GRID      = [3, 5, 10, 20, 50]      # Student‑t d.o.f grid for soft bma 
ETA_GRID    = [0.01, 0.05, 0.10, 0.15, 0.20,
               0.25, 0.30, 0.35]   # learning‑rates for MWU
DECAYS = np.arange(0.75, 1.00, 0.05)

# dynamic date anchors
TODAY             = pd.Timestamp.today().normalize()
FIRST_DATE        = pd.to_datetime(df_full["release_date"].min()).normalize()
TRAILING_START_12 = TODAY - pd.DateOffset(months=12)
TRAILING_START_24 = TODAY - pd.DateOffset(months=24)

# define stratified regimes of interest
REGIMES = {
    f"{FIRST_DATE:%Y‑%m} to 2007‑12 (pre‑GFC)" : (FIRST_DATE, "2007-12-31"),
    "2008‑01 to 2009‑12 (GFC)"                : ("2008-01-01", "2009-12-31"),
    "2010‑01‑2014‑12 (early‑expansion, post GFC)"             : ("2010-01-01", "2014-12-31"),
    "2015‑01‑2019‑12 (late‑expansion, post GFC)"              : ("2015-01-01", "2019-12-31"),
    "2020‑01 to 2022‑12 (COVID)"              : ("2020-01-01", "2022-12-31"),
    f"2023‑01 to {TODAY.date()} (post‑COVID)" : ("2023-01-01", TODAY),
    "Trailing 24‑months"                      : (TRAILING_START_24, TODAY),
    "Trailing 12‑months"                      : (TRAILING_START_12, TODAY)
}

In [ ]:
live_forecasts = []

# For each method we’ll store: 
#   eval_tables[model_id][panel_name] = eval_df
#   live_tables[model_id][panel_name] = live_df
#   oos_maps   [model_id][panel_name] = oos_map

eval_tables = defaultdict(dict)
live_tables = defaultdict(dict)
oos_maps     = defaultdict(dict)
actual_dir   = defaultdict(dict)

# And also build flat lists for later concatenation
_all_eval = []
_all_live = []

## 1.0: Static inverse-error

In [ ]:
# ---------------------------------------------------------------
#  INVERSE‑ERROR ENSEMBLE  (spec_id‑ready for later voting combo)
# ---------------------------------------------------------------

# -------------------- HELPERS ----------------------------------
def weight_vector(history: pd.DataFrame, economists: pd.Index, method: str) -> pd.Series:
    """Return inverse‑error (MSE / MAE) or equal weights."""
    if method == "equal_weight":
        w = pd.Series(1.0, index=economists)
    else:
        grp   = history[history["economist"].isin(economists)] \
                     .groupby("economist")["error"]
        score = grp.apply(lambda s: np.nanmean(s**2) if method == "inverse_mse"
                          else np.nanmean(np.abs(s)))
        w = 1.0 / (score + RIDGE)
    return w / w.sum()

# -------------------- BACKTEST ---------------------------------
def backtest_panel(panel: pd.DataFrame, label: str):
    """
    Walk‑forward evaluation over (window, method).
    Returns: eval_df, live_df, oos_map (keyed by spec_id)
    """
    dates   = np.sort(panel["release_date"].unique())
    eval_r, live_r, oos_map = [], [], {}

    for window, method in tqdm(product(CONT_WINDOWS, METHODS),
                               total=len(CONT_WINDOWS)*len(METHODS),
                               desc=f"{label} grid"):
        spec_id = f"inv_err_w{window}_m{method}"
        recs    = []

        # walk‑forward
        for idx in range(window, len(dates)):
            t_date   = dates[idx]
            hist_idx = dates[idx-window:idx]
            hist     = panel[panel["release_date"].isin(hist_idx)]
            econs    = hist.groupby("economist")["forecast"] \
                            .apply(lambda s: s.notna().all())
            econs    = econs[econs].index
            if econs.empty:
                continue

            w = weight_vector(hist, econs, method)

            cur = panel[(panel["release_date"] == t_date) &
                        (panel["economist"].isin(w.index))]
            f_t = cur.set_index("economist")["forecast"].dropna()
            w   = w.reindex(f_t.index).dropna()
            if w.empty:
                continue
            w /= w.sum()

            smart   = np.dot(w, f_t.loc[w.index])
            median  = panel.loc[panel["release_date"] == t_date, "median_forecast"].iloc[0]
            actual  = panel.loc[panel["release_date"] == t_date, "actual"].iloc[0]
            preddir = int(smart > median)

            recs.append((t_date, smart, median, actual, preddir))

        if not recs:
            continue

        # out‑of‑sample frame
        oos = pd.DataFrame(recs,
                           columns=["date", "smart", "median", "actual", "pred_dir"])
        oos_map[spec_id] = oos

        # live row if latest actual is still NA
        if pd.isna(oos.iloc[-1, 3]):
            last = oos.iloc[-1]
            live_r.append({
                "spec_id": spec_id,
                "panel":   label,
                "window":  window,
                "method":  method,
                "date":    last["date"],
                "smart":   last["smart"],
                "median":  last["median"],
                "pred_dir":last["pred_dir"]
            })

        # realised evaluation
        eval_df = oos.dropna(subset=["actual"]).copy()
        if eval_df.empty:
            continue

        eval_df["smart_err"]  = eval_df["smart"]  - eval_df["actual"]
        eval_df["median_err"] = eval_df["median"] - eval_df["actual"]
        eval_df["actual_dir"] = (eval_df["actual"] > eval_df["median"]).astype(int)

        obs      = len(eval_df)
        rmse_s   = np.sqrt((eval_df["smart_err"]**2).mean())
        rmse_m   = np.sqrt((eval_df["median_err"]**2).mean())
        diff     = eval_df["smart_err"]**2 - eval_df["median_err"]**2
        dm_p     = 2*(1 - stats.norm.cdf(abs(diff.mean()/diff.std(ddof=1)*np.sqrt(obs))))
        hits     = (eval_df["pred_dir"] == eval_df["actual_dir"]).sum()
        hit_rate = hits / obs
        binom_p  = stats.binomtest(hits, obs, .5).pvalue
        p1, p2   = eval_df["pred_dir"].mean(), eval_df["actual_dir"].mean()
        c_joint  = (eval_df["pred_dir"] & eval_df["actual_dir"]).mean()
        pt_p     = 2*(1 - stats.norm.cdf(abs((c_joint - p1*p2) /
                                             np.sqrt(p1*p2*(1-p1)*(1-p2)/obs))))

        eval_r.append({
            "spec_id":     spec_id,
            "panel":       label,
            "window":      window,
            "method":      method,
            "obs":         obs,
            "RMSE_smart":  rmse_s,
            "RMSE_median": rmse_m,
            "SmartBetter": int(rmse_s < rmse_m),
            "HitRate":     hit_rate,
            "Binom_p":     binom_p,
            "PT_p":        pt_p,
            "DM_p":        dm_p
        })

    # assemble DataFrames
    eval_df = pd.DataFrame(eval_r)
    live_df = pd.DataFrame(live_r)
    eval_df["model_id"] = "inv_err"
    live_df["model_id"] = "inv_err"
    return eval_df, live_df, oos_map

# =================== DRIVER & PRINTING =========================

# run & collect
for name, pnl in PANELS.items():
    ev, lv, om = backtest_panel(pnl, name)
    eval_tables["inv_err"][name] = ev
    live_tables["inv_err"][name] = lv
    oos_maps    ["inv_err"][name] = om
    _all_eval.append(ev)
    _all_live.append(lv)

    if om:
        realised = next(iter(om.values())).dropna(subset=["actual"])
        actual_dir["inv_err"][name] = (realised["actual"] > realised["median"]).astype(int).values
    else:
        actual_dir["inv_err"][name] = np.array([], dtype=int)

# formatting & key‑spec function
pd.set_option("display.float_format", "{:.3f}".format)

def print_key_specs(eval_df: pd.DataFrame, panel_name: str) -> None:
    low_rmse = eval_df.loc[eval_df["RMSE_smart"].idxmin()]
    high_hit = eval_df.loc[eval_df["HitRate"].idxmax()]
    robust = eval_df[(eval_df["DM_p"] < .10) & (eval_df["PT_p"] < .10)]
    if not robust.empty:
        r = robust.loc[robust["RMSE_smart"].idxmin()]
        rob_str = f"{r['spec_id']} (w={int(r['window'])}, m={r['method']})"
    else:
        rob_str = "None (DM_p & PT_p ≥ 0.10)"

    print(f"\n{panel_name} panel key specs:")
    print(f"  • Lowest RMSE    : {low_rmse['spec_id']} (w={int(low_rmse['window'])}, m={low_rmse['method']})")
    print(f"  • Highest HitRate: {high_hit['spec_id']} (w={int(high_hit['window'])}, m={high_hit['method']})")
    print(f"  • Robust Winner  : {rob_str}")

# print back‑test tables and key specs
print("\n=== Back‑test summary (COVID panel) ===")
print(eval_tables["inv_err"]["COVID"].to_string(index=False))
print_key_specs(eval_tables["inv_err"]["COVID"], "COVID")

print("\n=== Back‑test summary (Full panel) ===")
print(eval_tables["inv_err"]["Full"].to_string(index=False))
print_key_specs(eval_tables["inv_err"]["Full"], "Full")

# stratified diagnostics on best‑HitRate spec (FULL)
best = eval_tables["inv_err"]["Full"].loc[
    eval_tables["inv_err"]["Full"]["HitRate"].idxmax(), "spec_id"]
oos_best = oos_maps["inv_err"]["Full"][best]

def stratified_diagnostics(oos: pd.DataFrame, regimes=REGIMES):
    rows = []
    df = oos.dropna(subset=["actual"]).copy()
    if df.empty: return pd.DataFrame()
    df["smart_err"]  = df["smart"]  - df["actual"]
    df["median_err"] = df["median"] - df["actual"]
    df["pred_dir"]   = (df["smart"] > df["median"]).astype(int)
    df["actual_dir"] = (df["actual"] > df["median"]).astype(int)
    for lbl, (s,e) in REGIMES.items():
        sub = df[(df["date"]>=s)&(df["date"]<=e)]
        if sub.empty: continue
        obs = len(sub)
        rm_s = np.sqrt((sub["smart_err"]**2).mean())
        rm_m = np.sqrt((sub["median_err"]**2).mean())
        diff = sub["smart_err"]**2 - sub["median_err"]**2
        dm_p = 2*(1 - stats.norm.cdf(abs(diff.mean()/diff.std(ddof=1)*np.sqrt(obs))))
        hits = (sub["pred_dir"]==sub["actual_dir"]).sum()
        hr   = hits/obs
        rows.append({"Regime":lbl,"Obs":obs,"RMSE_smart":rm_s,"RMSE_median":rm_m,"HitRate":hr,"DM_p":dm_p})
    return pd.DataFrame(rows)

print(f"\n=== Stratified diagnostics (FULL • best spec {best}) ===")
tbl = stratified_diagnostics(oos_best)
print("No realised data." if tbl.empty else tbl.to_string(index=False))

# consolidated live forecasts
print("\n================ CONSOLIDATED LIVE FORECASTS ================\n")
for panel in ["COVID","Full"]:
    lv = live_tables["inv_err"][panel]
    if lv.empty: continue
    ev = eval_tables["inv_err"][panel]
    # choose specs
    sel = {
      "Lowest RMSE":     ev.loc[ev["RMSE_smart"].idxmin()],
      "Highest HitRate": ev.loc[ev["HitRate"].idxmax()]
    }
    rob = ev[(ev["DM_p"]<.10)&(ev["PT_p"]<.10)]
    if not rob.empty:
        sel["Robust Winner"] = rob.loc[rob["RMSE_smart"].idxmin()]
    # merge labels & print
    from collections import defaultdict
    lm, info = defaultdict(set), {}
    for lbl,row in sel.items():
        lm[row["spec_id"]].add(lbl); info[row["spec_id"]]=row
    for sid, labels in lm.items():
        row = lv[lv["spec_id"]==sid].iloc[-1]
        txt = " & ".join(sorted(labels))
        sig = "Beat" if row["pred_dir"] else "Miss"
        print(f"--- {panel} • {txt} ---")
        print(f"Date   : {row['date'].date()}")
        print(f"Smart  : {row['smart']:.1f} k")
        print(f"Median : {row['median']:.1f} k")
        print(f"Signal : {sig}  ({sid})\n")


COVID grid:   0%|          | 0/9 [00:00<?, ?it/s]

Full grid:   0%|          | 0/9 [00:00<?, ?it/s]


=== Back‑test summary (COVID panel) ===
                  spec_id panel  window       method  obs  RMSE_smart  RMSE_median  SmartBetter  HitRate  Binom_p  PT_p  DM_p model_id
  inv_err_w3_minverse_mse COVID       3  inverse_mse  227      73.452       73.173            0    0.467    0.353 0.326 0.758  inv_err
  inv_err_w3_minverse_mae COVID       3  inverse_mae  227      72.621       73.173            1    0.529    0.426 0.369 0.140  inv_err
 inv_err_w3_mequal_weight COVID       3 equal_weight  227      72.623       73.173            1    0.559    0.084 0.073 0.025  inv_err
  inv_err_w6_minverse_mse COVID       6  inverse_mse  224      72.642       72.955            1    0.513    0.738 0.679 0.678  inv_err
  inv_err_w6_minverse_mae COVID       6  inverse_mae  224      72.479       72.955            1    0.518    0.640 0.571 0.139  inv_err
 inv_err_w6_mequal_weight COVID       6 equal_weight  224      72.428       72.955            1    0.554    0.124 0.109 0.036  inv_err
 inv_err_w12_m

## 1.2 Exponentially Weighted Moving Average (EWMA)

In [ ]:
# ----------------------------------------------------------------
# EWMA helpers
# ----------------------------------------------------------------
def ewma_time_weights(window: int, decay: float) -> np.ndarray:
    """Exponentially‑decaying (oldest→newest) weights that sum to 1."""
    w = decay ** np.arange(window - 1, -1, -1, dtype=float)
    return w / w.sum()

# ----------------------------------------------------------------
# Back‑tester
# ----------------------------------------------------------------
def backtest_ewma(panel: pd.DataFrame,
                  windows=CONT_WINDOWS,
                  decays=DECAYS,
                  methods=METHODS,
                  ridge: float = RIDGE):
    """
    Walk‑forward grid search over (window, decay, method).
    Returns: eval_df, live_df, oos_map (keyed by spec_id)
    """
    pname   = getattr(panel, "name", "panel")
    dates   = np.sort(panel["release_date"].unique())
    eval_r  = []
    live_r  = []
    oos_map = {}

    for window, decay, method in tqdm(
        product(windows, decays, methods),
        total=len(windows)*len(decays)*len(methods),
        desc=f"{pname} grid"
    ):
        spec_id = f"ewma_w{window}_d{decay:.2f}_m{method}"
        recs    = []

        # walk‑forward
        for idx in range(window, len(dates)):
            t_date   = dates[idx]
            hist_idx = dates[idx-window:idx]
            hist     = panel[panel["release_date"].isin(hist_idx)]

            econs = (
                hist.groupby("economist")["forecast"]
                    .apply(lambda s: s.notna().all())
                    .pipe(lambda s: s[s].index)
            )
            if econs.empty:
                continue

            # economist weights
            if method == "equal_weight":
                w_econ = pd.Series(1.0, index=econs)
            else:
                t_w    = ewma_time_weights(window, decay)
                scores = {}
                for e in econs:
                    errs = (
                        hist.loc[hist["economist"] == e]
                            .sort_values("release_date")["error"]
                            .values
                    )
                    if len(errs) != window:
                        continue
                    # use MSE or MAE
                    scores[e] = np.sum(t_w * (errs**2 if method=="inverse_mse"
                                              else np.abs(errs)))
                if not scores:
                    continue
                w_econ = pd.Series({e: 1.0/(s+ridge) for e, s in scores.items()})
            w_econ /= w_econ.sum()

            # current‑month forecasts
            cur = panel[
                (panel["release_date"] == t_date) &
                (panel["economist"].isin(w_econ.index))
            ]
            f_t = cur.set_index("economist")["forecast"].dropna()
            w   = w_econ.reindex(f_t.index).dropna()
            if w.empty:
                continue
            w /= w.sum()

            smart   = float(np.dot(w, f_t.loc[w.index]))
            median  = float(panel.loc[panel["release_date"] == t_date, "median_forecast"].iloc[0])
            actual  = float(panel.loc[panel["release_date"] == t_date, "actual"].iloc[0])
            preddir = int(smart > median)

            recs.append((t_date, smart, median, actual, preddir))

        if not recs:
            continue

        # store out‑of‑sample frame
        oos_map[spec_id] = pd.DataFrame(
            recs,
            columns=["date","smart","median","actual","pred_dir"]
        )

        # live row if latest actual is still missing
        oos = oos_map[spec_id]
        if pd.isna(oos.iloc[-1, 3]):
            last = oos.iloc[-1]
            live_r.append({
                "spec_id": spec_id, "panel": pname,
                "window":  window,  "decay": decay, "method": method,
                "date":    last["date"], "smart": last["smart"],
                "median":  last["median"], "pred_dir": last["pred_dir"]
            })

        # realised evaluation
        df_eval = oos.dropna(subset=["actual"]).copy()
        if df_eval.empty:
            continue

        df_eval["smart_err"]  = df_eval["smart"]  - df_eval["actual"]
        df_eval["median_err"] = df_eval["median"] - df_eval["actual"]
        df_eval["actual_dir"] = (df_eval["actual"] > df_eval["median"]).astype(int)

        obs      = len(df_eval)
        rmse_s   = np.sqrt((df_eval["smart_err"]**2).mean())
        rmse_m   = np.sqrt((df_eval["median_err"]**2).mean())
        diff     = df_eval["smart_err"]**2 - df_eval["median_err"]**2
        dm_p     = 2*(1 - stats.norm.cdf(abs(diff.mean()/diff.std(ddof=1)*np.sqrt(obs))))
        hits     = (df_eval["pred_dir"] == df_eval["actual_dir"]).sum()
        hit_rate = hits/obs
        binom_p  = stats.binomtest(hits, obs, .5).pvalue
        p1, p2   = df_eval["pred_dir"].mean(), df_eval["actual_dir"].mean()
        c_joint  = (df_eval["pred_dir"] & df_eval["actual_dir"]).mean()
        pt_p     = 2*(1 - stats.norm.cdf(abs((c_joint - p1*p2) /
                                             np.sqrt(p1*p2*(1-p1)*(1-p2)/obs))))

        eval_r.append({
            "spec_id":      spec_id,
            "panel":        pname,
            "window":       window,
            "decay":        decay,
            "method":       method,
            "obs":          obs,
            "RMSE_smart":   rmse_s,
            "RMSE_median":  rmse_m,
            "SmartBetter":  int(rmse_s < rmse_m),
            "HitRate":      hit_rate,
            "Binom_p":      binom_p,
            "PT_p":         pt_p,
            "DM_p":         dm_p
        })

    # assemble DataFrames
    eval_df = pd.DataFrame(eval_r)
    live_df = pd.DataFrame(live_r)
    eval_df["model_id"] = "ewma"
    live_df["model_id"] = "ewma"
    return eval_df, live_df, oos_map

# ----------------------------------------------------------------
# Stratified diagnostics helper
# ----------------------------------------------------------------
def stratified_ewma(oos: pd.DataFrame, regimes=REGIMES):
    rows = []
    df = oos.dropna(subset=["actual"]).copy()
    if df.empty:
        return pd.DataFrame()

    df["smart_err"]  = df["smart"]  - df["actual"]
    df["median_err"] = df["median"] - df["actual"]
    df["pred_dir"]   = (df["smart"] > df["median"]).astype(int)
    df["actual_dir"] = (df["actual"] > df["median"]).astype(int)

    for lbl, (start, end) in regimes.items():
        sub = df[(df["date"] >= start) & (df["date"] <= end)]
        if sub.empty:
            continue
        obs      = len(sub)
        rm_s     = np.sqrt((sub["smart_err"]**2).mean())
        rm_m     = np.sqrt((sub["median_err"]**2).mean())
        diff     = sub["smart_err"]**2 - sub["median_err"]**2
        dm_p     = 2*(1 - stats.norm.cdf(abs(diff.mean()/diff.std(ddof=1)*np.sqrt(obs))))
        hits     = (sub["pred_dir"] == sub["actual_dir"]).sum()
        hit_rate = hits/obs

        rows.append({
            "Regime":      lbl,
            "Obs":         obs,
            "RMSE_smart":  rm_s,
            "RMSE_median": rm_m,
            "SmartBetter": int(rm_s < rm_m),
            "HitRate":     hit_rate,
            "DM_p":        dm_p
        })

    return pd.DataFrame(rows)

# ----------------------------------------------------------------
# Driver: run back‑test, store and print results
# ----------------------------------------------------------------
for name, pnl in PANELS.items():
    pnl.name = name
    ev, lv, om = backtest_ewma(pnl)
    eval_tables["ewma"][name] = ev
    live_tables["ewma"][name] = lv
    oos_maps    ["ewma"][name] = om
    _all_eval.append(ev)
    _all_live.append(lv)

    if om:
        realised = next(iter(om.values())).dropna(subset=["actual"])
        actual_dir["ewma"][name] = (realised["actual"] > realised["median"]).astype(int).values
    else:
        actual_dir["ewma"][name] = np.array([], dtype=int)

# formatting & key‑spec function
pd.set_option("display.float_format", "{:.3f}".format)
def print_key_specs_ewma(df: pd.DataFrame, panel_name: str):
    low = df.loc[df["RMSE_smart"].idxmin()]
    high= df.loc[df["HitRate"].idxmax()]
    rob = df[(df["DM_p"]<.10)&(df["PT_p"]<.10)]
    if not rob.empty:
        r = rob.loc[rob["RMSE_smart"].idxmin()]
        rob_str = f"{r['spec_id']} (w={int(r['window'])}, d={r['decay']:.2f}, m={r['method']})"
    else:
        rob_str = "None (DM_p & PT_p ≥ 0.10)"
    print(f"\n{panel_name} panel key specs:")
    print(f"  • Lowest RMSE    : {low['spec_id']} (w={int(low['window'])}, d={low['decay']:.2f}, m={low['method']})")
    print(f"  • Highest HitRate: {high['spec_id']} (w={int(high['window'])}, d={high['decay']:.2f}, m={high['method']})")
    print(f"  • Robust Winner  : {rob_str}")

# print back‑test tables + key specs
print("\n=== Back‑test summary (COVID panel) ===")
print(eval_tables["ewma"]["COVID"].to_string(index=False))
print_key_specs_ewma(eval_tables["ewma"]["COVID"], "COVID")

print("\n=== Back‑test summary (Full panel) ===")
print(eval_tables["ewma"]["Full"].to_string(index=False))
print_key_specs_ewma(eval_tables["ewma"]["Full"], "Full")

# stratified diagnostics for best‑RMSE spec
best = eval_tables["ewma"]["Full"].loc[
    eval_tables["ewma"]["Full"]["HitRate"].idxmax(),"spec_id"
]
oos_best = oos_maps["ewma"]["Full"][best]

print(f"\n=== Stratified diagnostics (FULL • best spec {best}) ===")
tbl = stratified_ewma(oos_best)
print("No realised data." if tbl.empty else tbl.to_string(index=False))

# consolidated live forecasts
print("\n================ CONSOLIDATED LIVE FORECASTS ================\n")
for panel in ["COVID","Full"]:
    lv = live_tables["ewma"][panel]
    if lv.empty: continue
    ev = eval_tables["ewma"][panel]
    sel = {
      "Lowest RMSE":     ev.loc[ev["RMSE_smart"].idxmin()],
      "Highest HitRate": ev.loc[ev["HitRate"].idxmax()]
    }
    rob = ev[(ev["DM_p"]<.10)&(ev["PT_p"]<.10)]
    if not rob.empty:
        sel["Robust Winner"] = rob.loc[rob["RMSE_smart"].idxmin()]
    from collections import defaultdict
    lm, info = defaultdict(set), {}
    for lbl,row in sel.items():
        lm[row["spec_id"]].add(lbl); info[row["spec_id"]] = row
    for sid, labels in lm.items():
        row = lv[lv["spec_id"]==sid].iloc[-1]
        txt = " & ".join(sorted(labels))
        sig = "Beat" if row["pred_dir"] else "Miss"
        print(f"--- {panel} • {txt} ---")
        print(f"Date   : {row['date'].date()}")
        print(f"Smart  : {row['smart']:.1f} k")
        print(f"Median : {row['median']:.1f} k")
        print(f"Signal : {sig}  ({sid})\n")


COVID grid:   0%|          | 0/45 [00:00<?, ?it/s]

Full grid:   0%|          | 0/45 [00:00<?, ?it/s]


=== Back‑test summary (COVID panel) ===
                     spec_id panel  window  decay       method  obs  RMSE_smart  RMSE_median  SmartBetter  HitRate  Binom_p  PT_p  DM_p model_id
  ewma_w3_d0.75_minverse_mse COVID       3  0.750  inverse_mse  227      72.280       73.173            1    0.542    0.232 0.199 0.141     ewma
  ewma_w3_d0.75_minverse_mae COVID       3  0.750  inverse_mae  227      72.257       73.173            1    0.533    0.353 0.311 0.041     ewma
 ewma_w3_d0.75_mequal_weight COVID       3  0.750 equal_weight  227      72.623       73.173            1    0.559    0.084 0.073 0.025     ewma
  ewma_w3_d0.80_minverse_mse COVID       3  0.800  inverse_mse  227      72.294       73.173            1    0.546    0.184 0.155 0.145     ewma
  ewma_w3_d0.80_minverse_mae COVID       3  0.800  inverse_mae  227      72.268       73.173            1    0.533    0.353 0.311 0.041     ewma
 ewma_w3_d0.80_mequal_weight COVID       3  0.800 equal_weight  227      72.623       73.

## 1.3 Rolling-window soft-BMA with Student-t plug-in likelihood

In [ ]:
# ------------------ HELPER FUNCTIONS --------------------------
def soft_bma_weights(err_dict: dict[str, np.ndarray], nu: int) -> pd.Series:
    """
    Compute soft‑BMA weights under a Student‑t error model.
    """
    ll = {}
    for econ, e in err_dict.items():
        if e.size == 0 or np.std(e, ddof=1) == 0.0:
            continue
        σ = np.std(e, ddof=1)
        ll[econ] = stats.t.logpdf(e, df=nu, loc=0.0, scale=σ).sum()
    if not ll:
        return pd.Series(dtype=float)
    vals = np.array(list(ll.values()))
    w    = np.exp(vals - vals.max())
    return pd.Series(w, index=list(ll.keys())) / w.sum()

# -------------------- BACKTEST SOFT‑BMA ------------------------
def backtest_soft_bma(
    panel: pd.DataFrame,
    windows: list[int] = CONT_WINDOWS,
    nus: list[int]     = NU_GRID
):
    """
    Walk‑forward grid search over (window, nu).
    Returns: eval_df, live_df, oos_map (keyed by spec_id).
    """
    p_name    = getattr(panel, "name", "panel")
    dates     = np.sort(panel["release_date"].unique())
    eval_rows = []
    live_rows = []
    oos_map   = {}

    for W, nu in tqdm(
        product(windows, nus),
        total=len(windows)*len(nus),
        desc=f"{p_name} grid"
    ):
        spec_id = f"soft_bma_w{W}_nu{nu}"
        recs    = []

        # — walk forward —
        for idx in range(W, len(dates)):
            t_date = dates[idx]
            hist   = panel[panel["release_date"].isin(dates[idx-W:idx])]

            # economists with full history
            econs = (
                hist.groupby("economist")["forecast"]
                    .apply(lambda s: s.notna().all())
                    .pipe(lambda s: s[s].index)
            )
            if econs.empty:
                continue

            # build error dict
            err_dict = {}
            for econ in econs:
                errs = (
                    hist[hist["economist"] == econ]
                        .sort_values("release_date")["error"]
                        .values
                )
                if len(errs) == W:
                    err_dict[econ] = errs
            if not err_dict:
                continue

            # compute weights
            w_econ = soft_bma_weights(err_dict, nu)
            if w_econ.empty:
                continue

            # align with current forecasts
            cur = panel[
                (panel["release_date"] == t_date) &
                (panel["economist"].isin(w_econ.index))
            ]
            f_t = cur.set_index("economist")["forecast"].dropna()
            w   = w_econ.reindex(f_t.index).dropna()
            if w.empty:
                continue
            w /= w.sum()

            # form ensemble
            smart    = float(np.dot(w, f_t.loc[w.index]))
            median   = float(panel.loc[
                panel["release_date"] == t_date, "median_forecast"
            ].iloc[0])
            actual   = float(panel.loc[
                panel["release_date"] == t_date, "actual"
            ].iloc[0])
            pred_dir = int(smart > median)
            recs.append((t_date, smart, median, actual, pred_dir))

        if not recs:
            continue

        # store out‑of‑sample DataFrame
        oos_map[spec_id] = pd.DataFrame(
            recs,
            columns=["date","smart","median","actual","pred_dir"]
        )

        # record live row if latest actual is missing
        last = oos_map[spec_id].iloc[-1]
        if pd.isna(last["actual"]):
            live_rows.append({
                "spec_id": spec_id, "panel": p_name,
                "window":  W,       "nu":    nu,
                "date":    last["date"],
                "smart":   last["smart"],
                "median":  last["median"],
                "pred_dir": last["pred_dir"]
            })

        # realised evaluation
        df_eval = oos_map[spec_id].dropna(subset=["actual"]).copy()
        if df_eval.empty:
            continue

        df_eval["smart_err"]  = df_eval["smart"]  - df_eval["actual"]
        df_eval["median_err"] = df_eval["median"] - df_eval["actual"]
        df_eval["pred_dir"]   = (df_eval["smart"]  > df_eval["median"]).astype(int)
        df_eval["actual_dir"] = (df_eval["actual"] > df_eval["median"]).astype(int)

        obs      = len(df_eval)
        rmse_s   = np.sqrt((df_eval["smart_err"]**2).mean())
        rmse_m   = np.sqrt((df_eval["median_err"]**2).mean())
        diff     = df_eval["smart_err"]**2 - df_eval["median_err"]**2
        dm_p     = 2*(1 - stats.norm.cdf(
                        abs(diff.mean()/diff.std(ddof=1)*np.sqrt(obs))
                     ))
        hits     = (df_eval["pred_dir"] == df_eval["actual_dir"]).sum()
        hit_rate = hits/obs
        binom_p  = stats.binomtest(hits, obs, .5).pvalue
        p1, p2   = df_eval["pred_dir"].mean(), df_eval["actual_dir"].mean()
        c_joint  = (df_eval["pred_dir"] & df_eval["actual_dir"]).mean()
        pt_p     = 2*(1 - stats.norm.cdf(
                        abs((c_joint - p1*p2) /
                            np.sqrt(p1*p2*(1-p1)*(1-p2)/obs))
                     ))

        eval_rows.append({
            "spec_id":      spec_id,
            "panel":        p_name,
            "window":       W,
            "nu":           nu,
            "obs":          obs,
            "RMSE_smart":   rmse_s,
            "RMSE_median":  rmse_m,
            "SmartBetter":  int(rmse_s < rmse_m),
            "HitRate":      hit_rate,
            "Binom_p":      binom_p,
            "PT_p":         pt_p,
            "DM_p":         dm_p
        })

    # assemble DataFrames
    eval_df = pd.DataFrame(eval_rows)
    live_df = pd.DataFrame(live_rows)
    eval_df["model_id"] = "soft_bma"
    live_df["model_id"] = "soft_bma"
    return eval_df, live_df, oos_map

# ---------------- STRATIFIED DIAGNOSTICS -----------------------
def stratified_soft_bma(oos: pd.DataFrame, regimes: dict = REGIMES) -> pd.DataFrame:
    rows = []
    df = oos.dropna(subset=["actual"]).copy()
    if df.empty:
        return pd.DataFrame()

    df["smart_err"]  = df["smart"]  - df["actual"]
    df["median_err"] = df["median"] - df["actual"]
    df["pred_dir"]   = (df["smart"] > df["median"]).astype(int)
    df["actual_dir"] = (df["actual"] > df["median"]).astype(int)

    for lbl, (start, end) in regimes.items():
        sub = df[(df["date"] >= start) & (df["date"] <= end)]
        if sub.empty:
            continue
        obs      = len(sub)
        rm_s     = np.sqrt((sub["smart_err"]**2).mean())
        rm_m     = np.sqrt((sub["median_err"]**2).mean())
        diff     = sub["smart_err"]**2 - sub["median_err"]**2
        dm_p     = 2*(1 - stats.norm.cdf(
                        abs(diff.mean()/diff.std(ddof=1)*np.sqrt(obs))
                     ))
        hits     = (sub["pred_dir"] == sub["actual_dir"]).sum()
        hit_rate = hits/obs
        binom_p  = stats.binomtest(hits, obs, .5).pvalue
        p1, p2   = sub["pred_dir"].mean(), sub["actual_dir"].mean()
        c_joint  = (sub["pred_dir"] & sub["actual_dir"]).mean()
        pt_p     = 2*(1 - stats.norm.cdf(
                        abs((c_joint - p1*p2) /
                            np.sqrt(p1*p2*(1-p1)*(1-p2)/obs))
                     ))

        rows.append({
            "Regime":      lbl,
            "Obs":         obs,
            "RMSE_smart":  rm_s,
            "RMSE_median": rm_m,
            "SmartBetter": int(rm_s < rm_m),
            "HitRate":     hit_rate,
            "Binom_p":     binom_p,
            "PT_p":        pt_p,
            "DM_p":        dm_p
        })

    return pd.DataFrame(rows)

# ------------------------ DRIVER -------------------------------
for name, pnl in PANELS.items():
    pnl.name = name
    ev, lv, om = backtest_soft_bma(pnl)
    eval_tables["soft_bma"][name] = ev
    live_tables["soft_bma"][name] = lv
    oos_maps   ["soft_bma"][name] = om

    _all_eval.append(ev)
    _all_live.append(lv)

    if om:
        realised = next(iter(om.values())).dropna(subset=["actual"])
        actual_dir["soft_bma"][name] = (
            (realised["actual"] > realised["median"])
            .astype(int)
            .values
        )
    else:
        actual_dir["soft_bma"][name] = np.array([], dtype=int)

# ---- formatting & key‑spec summary ----
pd.set_option("display.float_format", "{:.3f}".format)
def print_key_specs_soft(df: pd.DataFrame, panel_name: str) -> None:
    low  = df.loc[df["RMSE_smart"].idxmin()]
    high = df.loc[df["HitRate"].idxmax()]
    rob  = df[(df["DM_p"] < .10) & (df["PT_p"] < .10)]
    if not rob.empty:
        r = rob.loc[rob["RMSE_smart"].idxmin()]
        rob_str = f"{r['spec_id']} (w={int(r['window'])}, nu={int(r['nu'])})"
    else:
        rob_str = "None (DM_p & PT_p ≥ 0.10)"

    print(f"\n{panel_name} panel key specs:")
    print(f"  • Lowest RMSE    : {low['spec_id']} (w={int(low['window'])}, nu={int(low['nu'])})")
    print(f"  • Highest HitRate: {high['spec_id']} (w={int(high['window'])}, nu={int(high['nu'])})")
    print(f"  • Robust Winner  : {rob_str}")

# ---- back‑test tables + key specs ----
print("\n=== Back‑test summary (COVID panel) ===")
print(eval_tables["soft_bma"]["COVID"].to_string(index=False))
print_key_specs_soft(eval_tables["soft_bma"]["COVID"], "COVID")

print("\n=== Back‑test summary (Full panel) ===")
print(eval_tables["soft_bma"]["Full"].to_string(index=False))
print_key_specs_soft(eval_tables["soft_bma"]["Full"], "Full")

# ---- stratified diagnostics on best‑RMSE spec (FULL) ----
best_spec = eval_tables["soft_bma"]["Full"].loc[
    eval_tables["soft_bma"]["Full"]["HitRate"].idxmax(), "spec_id"
]
print(f"\n=== Stratified diagnostics (FULL • best spec {best_spec}) ===")
st_tbl = stratified_soft_bma(oos_maps["soft_bma"]["Full"][best_spec])
print("No realised data." if st_tbl.empty else st_tbl.to_string(index=False))

# ---- consolidated live forecasts ----
print("\n================ CONSOLIDATED LIVE FORECASTS ================\n")
for panel in ["COVID", "Full"]:
    lv = live_tables["soft_bma"][panel]
    if lv.empty:
        continue
    ev = eval_tables["soft_bma"][panel]

    # pick specs by criterion
    sel = {
        "Lowest RMSE":     ev.loc[ev["RMSE_smart"].idxmin()],
        "Highest HitRate": ev.loc[ev["HitRate"].idxmax()]
    }
    rob = ev[(ev["DM_p"] < .10) & (ev["PT_p"] < .10)]
    if not rob.empty:
        sel["Robust Winner"] = rob.loc[rob["RMSE_smart"].idxmin()]

    # merge labels → consolidated output
    from collections import defaultdict
    label_map, info = defaultdict(set), {}
    for lbl, row in sel.items():
        label_map[row["spec_id"]].add(lbl)
        info[row["spec_id"]] = row

    for sid, labels in label_map.items():
        row = lv[lv["spec_id"] == sid].iloc[-1]
        txt = " & ".join(sorted(labels))
        sig = "Beat" if row["pred_dir"] else "Miss"

        print(f"--- {panel} • {txt} ---")
        print(f"Date   : {row['date'].date()}")
        print(f"Smart  : {row['smart']:.1f} k")
        print(f"Median : {row['median']:.1f} k")
        print(f"Signal : {sig}  ({sid})\n")


COVID grid:   0%|          | 0/15 [00:00<?, ?it/s]

Full grid:   0%|          | 0/15 [00:00<?, ?it/s]


=== Back‑test summary (COVID panel) ===
          spec_id panel  window  nu  obs  RMSE_smart  RMSE_median  SmartBetter  HitRate  Binom_p  PT_p  DM_p model_id
  soft_bma_w3_nu3 COVID       3   3  227      72.673       73.173            1    0.515    0.691 0.628 0.524 soft_bma
  soft_bma_w3_nu5 COVID       3   5  227      72.658       73.173            1    0.529    0.426 0.377 0.518 soft_bma
 soft_bma_w3_nu10 COVID       3  10  227      72.645       73.173            1    0.524    0.507 0.452 0.514 soft_bma
 soft_bma_w3_nu20 COVID       3  20  227      72.639       73.173            1    0.537    0.288 0.253 0.514 soft_bma
 soft_bma_w3_nu50 COVID       3  50  227      72.636       73.173            1    0.533    0.353 0.313 0.514 soft_bma
  soft_bma_w6_nu3 COVID       6   3  224      72.040       72.955            1    0.554    0.124 0.105 0.306 soft_bma
  soft_bma_w6_nu5 COVID       6   5  224      72.038       72.955            1    0.558    0.095 0.078 0.307 soft_bma
 soft_bma_w6_nu

## 1.4 Multiplicative Weights Update (MWU)

In [ ]:
# ------------------ BACKTEST MWU ------------------------------
def backtest_mwu(
    panel: pd.DataFrame,
    windows: list[int] = CONT_WINDOWS,
    etas:    list[float] = ETA_GRID,
    ridge:   float       = RIDGE
):
    """
    Walk‑forward grid over (window, eta) for MWU.
    Returns: eval_df, live_df, oos_map keyed by spec_id.
    """
    pname     = getattr(panel, "name", "panel")
    dates     = np.sort(panel["release_date"].unique())
    # initial equal weights over all economists
    econs_all = np.sort(panel["economist"].dropna().unique())
    w0        = pd.Series(1.0 / len(econs_all), index=econs_all)

    eval_rows = []
    live_rows = []
    oos_map   = {}

    for W, eta in tqdm(
        product(windows, etas),
        total=len(windows) * len(etas),
        desc=f"{pname} grid"
    ):
        spec_id = f"mwu_w{W}_eta{eta:.3f}"
        w       = w0.copy()
        recs    = []

        # walk‑forward
        for idx, d in enumerate(dates):
            cur = panel[panel["release_date"] == d]

            # need W prior forecasts
            if idx < W:
                active = pd.Index([])
            else:
                hist = panel[panel["release_date"].isin(dates[idx-W:idx])]
                active = (
                    hist.groupby("economist")["forecast"]
                        .apply(lambda s: s.notna().all())
                        .pipe(lambda s: s[s].index)
                )

            if active.empty:
                continue

            # zero out inactive & renormalize
            w.loc[~w.index.isin(active)] = 0.0
            if w.sum() == 0:
                w.loc[active] = 1.0
            w /= w.sum()

            # form today's forecast if ≥2 available
            f_t   = cur.set_index("economist")["forecast"].reindex(w.index)
            avail = f_t.notna()
            if avail.sum() >= 2:
                w_av = w[avail].copy()
                if w_av.sum() == 0:
                    w_av[:] = 1.0
                w_av /= w_av.sum()
                smart  = float(np.dot(w_av, f_t[avail]))
                median = float(cur["median_forecast"].iloc[0])
                actual = float(cur["actual"].iloc[0])
                recs.append((d, smart, median, actual, int(smart > median)))

            # update weights after seeing actual
            if pd.notna(cur["actual"].iloc[0]):
                a_val = cur["actual"].iloc[0]
                loss  = (f_t - a_val).pow(2).fillna(0.0) + ridge
                w    *= np.exp(-eta * loss)
                if w.sum() == 0:
                    w.loc[active] = 1.0
                w    /= w.sum()

        if not recs:
            continue

        # out‑of‑sample frame
        oos = pd.DataFrame(recs, columns=["date","smart","median","actual","pred_dir"])
        oos_map[spec_id] = oos

        # live row if latest actual is missing
        last = oos.iloc[-1]
        if pd.isna(last["actual"]):
            live_rows.append({
                "spec_id": spec_id, "panel": pname,
                "window":   W,       "eta":   eta,
                "date":     last["date"],
                "smart":    last["smart"],
                "median":   last["median"],
                "pred_dir": last["pred_dir"]
            })

        # realised evaluation
        df_eval = oos.dropna(subset=["actual"]).copy()
        if df_eval.empty:
            continue

        df_eval["smart_err"]  = df_eval["smart"]  - df_eval["actual"]
        df_eval["median_err"] = df_eval["median"] - df_eval["actual"]
        df_eval["actual_dir"] = (df_eval["actual"] > df_eval["median"]).astype(int)

        obs      = len(df_eval)
        rmse_s   = np.sqrt((df_eval["smart_err"]**2).mean())
        rmse_m   = np.sqrt((df_eval["median_err"]**2).mean())
        diff     = df_eval["smart_err"]**2 - df_eval["median_err"]**2
        dm_p     = 2 * (1 - stats.norm.cdf(abs(diff.mean() / diff.std(ddof=1) * np.sqrt(obs))))
        hits     = (df_eval["pred_dir"] == df_eval["actual_dir"]).sum()
        hit_rate = hits / obs
        binom_p  = stats.binomtest(hits, obs, .5).pvalue
        p1, p2   = df_eval["pred_dir"].mean(), df_eval["actual_dir"].mean()
        c_joint  = (df_eval["pred_dir"] & df_eval["actual_dir"]).mean()
        pt_p     = 2 * (1 - stats.norm.cdf(abs((c_joint - p1*p2) /
                                               np.sqrt(p1*p2*(1-p1)*(1-p2)/obs))))

        eval_rows.append({
            "spec_id":      spec_id,
            "panel":        pname,
            "window":       W,
            "eta":          eta,
            "obs":          obs,
            "RMSE_smart":   rmse_s,
            "RMSE_median":  rmse_m,
            "SmartBetter":  int(rmse_s < rmse_m),
            "HitRate":      hit_rate,
            "Binom_p":      binom_p,
            "PT_p":         pt_p,
            "DM_p":         dm_p
        })

    eval_df = pd.DataFrame(eval_rows)
    live_df = pd.DataFrame(live_rows)
    eval_df["model_id"] = "mwu"
    live_df["model_id"] = "mwu"
    return eval_df, live_df, oos_map

# ---------------- STRATIFIED DIAGNOSTICS -----------------------
def stratified_mwu(oos: pd.DataFrame, regimes=REGIMES) -> pd.DataFrame:
    rows = []
    df  = oos.dropna(subset=["actual"]).copy()
    if df.empty:
        return pd.DataFrame()

    df["smart_err"]  = df["smart"]  - df["actual"]
    df["median_err"] = df["median"] - df["actual"]
    df["pred_dir"]   = (df["smart"] > df["median"]).astype(int)
    df["actual_dir"] = (df["actual"] > df["median"]).astype(int)

    for lbl, (start, end) in regimes.items():
        sub = df[(df["date"] >= start) & (df["date"] <= end)]
        if sub.empty:
            continue
        obs      = len(sub)
        rm_s     = np.sqrt((sub["smart_err"]**2).mean())
        rm_m     = np.sqrt((sub["median_err"]**2).mean())
        diff     = sub["smart_err"]**2 - sub["median_err"]**2
        dm_p     = 2 * (1 - stats.norm.cdf(abs(diff.mean() / diff.std(ddof=1) * np.sqrt(obs))))
        hits     = (sub["pred_dir"] == sub["actual_dir"]).sum()
        hit_rate = hits / obs
        binom_p  = stats.binomtest(hits, obs, .5).pvalue
        p1, p2   = sub["pred_dir"].mean(), sub["actual_dir"].mean()
        c_joint  = (sub["pred_dir"] & sub["actual_dir"]).mean()
        pt_p     = 2 * (1 - stats.norm.cdf(abs((c_joint - p1*p2) /
                                               np.sqrt(p1*p2*(1-p1)*(1-p2)/obs))))

        rows.append({
            "Regime":      lbl,
            "Obs":         obs,
            "RMSE_smart":  rm_s,
            "RMSE_median": rm_m,
            "SmartBetter": int(rm_s < rm_m),
            "HitRate":     hit_rate,
            "Binom_p":     binom_p,
            "PT_p":        pt_p,
            "DM_p":        dm_p
        })

    return pd.DataFrame(rows)

# --------------------- DRIVER & PRINTING ----------------------
for name, pnl in PANELS.items():
    pnl.name = name
    ev, lv, om = backtest_mwu(pnl)
    eval_tables["mwu"][name] = ev
    live_tables["mwu"][name] = lv
    oos_maps   ["mwu"][name] = om
    _all_eval.append(ev)
    _all_live.append(lv)
    if om:
        realised = next(iter(om.values())).dropna(subset=["actual"])
        actual_dir["mwu"][name] = (realised["actual"] > realised["median"]).astype(int).values
    else:
        actual_dir["mwu"][name] = np.array([], dtype=int)

pd.set_option("display.float_format", "{:.3f}".format)

def print_key_specs_mwu(df: pd.DataFrame, panel_name: str) -> None:
    low  = df.loc[df["RMSE_smart"].idxmin()]
    high = df.loc[df["HitRate"].idxmax()]
    rob  = df[(df["DM_p"] < .10) & (df["PT_p"] < .10)]
    if not rob.empty:
        r = rob.loc[rob["RMSE_smart"].idxmin()]
        rob_str = f"{r['spec_id']} (w={int(r['window'])}, η={r['eta']:.3f})"
    else:
        rob_str = "None (DM_p & PT_p ≥ 0.10)"

    print(f"\n{panel_name} panel key specs:")
    print(f"  • Lowest RMSE    : {low['spec_id']} (w={int(low['window'])}, η={low['eta']:.3f})")
    print(f"  • Highest HitRate: {high['spec_id']} (w={int(high['window'])}, η={high['eta']:.3f})")
    print(f"  • Robust Winner  : {rob_str}")

# back‑test tables + key specs
print("\n=== Back‑test summary (COVID panel) ===")
print(eval_tables["mwu"]["COVID"].to_string(index=False))
print_key_specs_mwu(eval_tables["mwu"]["COVID"], "COVID")

print("\n=== Back‑test summary (Full panel) ===")
print(eval_tables["mwu"]["Full"].to_string(index=False))
print_key_specs_mwu(eval_tables["mwu"]["Full"], "Full")

# stratified diagnostics on best‑RMSE spec (Full)
best_spec = eval_tables["mwu"]["Full"].loc[
    eval_tables["mwu"]["Full"]["HitRate"].idxmax(), "spec_id"
]
print(f"\n=== Stratified diagnostics (FULL • best spec {best_spec}) ===")
st_tbl = stratified_mwu(oos_maps["mwu"]["Full"][best_spec])
print("No realised data." if st_tbl.empty else st_tbl.to_string(index=False))

# consolidated live forecasts
print("\n================ CONSOLIDATED LIVE FORECASTS ================\n")
for panel in ["COVID", "Full"]:
    lv = live_tables["mwu"][panel]
    if lv.empty:
        continue
    ev = eval_tables["mwu"][panel]

    selections = {
        "Lowest RMSE"     : ev.loc[ev["RMSE_smart"].idxmin()],
        "Highest HitRate" : ev.loc[ev["HitRate"].idxmax()]
    }
    rob = ev[(ev["DM_p"] < .10) & (ev["PT_p"] < .10)]
    if not rob.empty:
        selections["Robust Winner"] = rob.loc[rob["RMSE_smart"].idxmin()]

    from collections import defaultdict
    label_map, spec_info = defaultdict(set), {}
    for lbl, row in selections.items():
        label_map[row["spec_id"]].add(lbl)
        spec_info[row["spec_id"]] = row

    for sid, labels in label_map.items():
        row = lv[lv["spec_id"] == sid].iloc[-1]
        txt = " & ".join(sorted(labels))
        sig = "Beat" if row["pred_dir"] else "Miss"
        print(f"--- {panel} • {txt} ---")
        print(f"Date   : {row['date'].date()}")
        print(f"Smart  : {row['smart']:.1f} k")
        print(f"Median : {row['median']:.1f} k")
        print(f"Signal : {sig}  ({sid})\n")


COVID grid:   0%|          | 0/24 [00:00<?, ?it/s]

Full grid:   0%|          | 0/24 [00:00<?, ?it/s]


=== Back‑test summary (COVID panel) ===
         spec_id panel  window   eta  obs  RMSE_smart  RMSE_median  SmartBetter  HitRate  Binom_p  PT_p  DM_p model_id
 mwu_w3_eta0.010 COVID       3 0.010  227      83.208       73.173            0    0.515    0.691 0.651 0.003      mwu
 mwu_w3_eta0.050 COVID       3 0.050  227      78.671       73.173            0    0.515    0.691 0.653 0.005      mwu
 mwu_w3_eta0.100 COVID       3 0.100  227      78.575       73.173            0    0.511    0.791 0.748 0.005      mwu
 mwu_w3_eta0.150 COVID       3 0.150  227      81.589       73.173            0    0.454    0.184 0.151 0.000      mwu
 mwu_w3_eta0.200 COVID       3 0.200  227      80.931       73.173            0    0.480    0.596 0.548 0.000      mwu
 mwu_w3_eta0.250 COVID       3 0.250  227      81.499       73.173            0    0.467    0.353 0.313 0.000      mwu
 mwu_w3_eta0.300 COVID       3 0.300  227      80.673       73.173            0    0.476    0.507 0.458 0.000      mwu
 mwu_w3

### 1.5 Ensemble majority based voting of smart forecasts for directional edge

TODO: is there a smarter/more informed way we can do this? 

In [90]:
# concatenate everything into one table
master_eval = pd.concat(_all_eval, ignore_index=True)
master_live = pd.concat(_all_live, ignore_index=True)

In [ ]:
# # in-sample, done via smart forecast averaging 

# # -------------------------------------------------------------
# # Majority‑vote ensemble search  (k = 3, 5)
# #   • choose best combo on  FULL, trailing‑24 m, trailing‑12 m
# #   • show full stratified directional diagnostics
# #   • finish with a summary table + majority “verdict”
# # -------------------------------------------------------------

# # ──────────────────────  SETTINGS  ────────────────────────────
# TODAY   = pd.Timestamp.today().normalize()
# WINDOWS = {
#     "FULL" : None,
#     "T24M" : TODAY - pd.DateOffset(months=24),
#     "T12M" : TODAY - pd.DateOffset(months=12),
#     "T6M" : TODAY - pd.DateOffset(months=6),
# }

# # REGIMES was defined earlier; re‑use the same dict.
# # If not present in your notebook, paste the REGIMES definition here.

# # ─────────── 1) candidate pool (unchanged)  ───────────────────
# pool = set()
# for model_id, panel_map in eval_tables.items():
#     for panel in ["COVID", "Full"]:
#         df = panel_map.get(panel)
#         if df is None or df.empty:
#             continue
#         pool.add(df.loc[df["RMSE_smart"].idxmin(), "spec_id"])  # lowest RMSE
#         pool.add(df.loc[df["HitRate"].idxmax(),    "spec_id"])  # highest HR
#         rob = df[(df["DM_p"] < .10) & (df["PT_p"] < .10)]
#         if not rob.empty:
#             pool.add(rob.loc[rob["RMSE_smart"].idxmin(), "spec_id"])

# pool = sorted(pool)
# print(f"Total unique candidate specs selected: {len(pool)}")

# # ─────────── 2) collect FULL‑panel OOS frames  ────────────────
# oos_full = {}
# for mdl, panel_map in oos_maps.items():
#     oos_full.update(panel_map.get("Full", {}))

# pool = [s for s in pool if s in oos_full]
# assert len(pool) >= 3, "Need ≥3 viable specs with Full‑panel OOS"
# print(f"Usable specs with Full‑panel OOS: {len(pool)}\n")

# # ─────────── 3) helpers  ──────────────────────────────────────
# def merged_oos(combo):
#     """Return merged DataFrame (ens, median, actual) for entire history."""
#     k   = len(combo)
#     dfs = []
#     for i, sid in enumerate(combo):
#         tmp = (oos_full[sid][["date", "smart", "median", "actual"]]
#                .rename(columns={"smart":  f"smart_{i}",
#                                 "median": f"median_{i}",
#                                 "actual": f"actual_{i}"}))
#         dfs.append(tmp)
#     df = dfs[0]
#     for d in dfs[1:]:
#         df = df.merge(d, on="date")

#     df["median"] = df["median_0"]
#     df["actual"] = df["actual_0"]
#     df["ens"]    = df[[f"smart_{i}" for i in range(k)]].mean(axis=1)
#     return df[["date", "ens", "median", "actual"]]

# def dir_metrics(df):
#     """Directional hit‑rate + Binom/PT p‑values on realised part of df."""
#     realised = df.dropna(subset=["actual"]).copy()
#     realised["pred_dir"]   = (realised["ens"]    > realised["median"]).astype(int)
#     realised["actual_dir"] = (realised["actual"] > realised["median"]).astype(int)

#     hits = (realised["pred_dir"] == realised["actual_dir"]).sum()
#     n    = len(realised)
#     if n == 0:
#         return np.nan, n, np.nan, np.nan
#     hr   = hits / n
#     binom_p = stats.binomtest(hits, n, .5).pvalue
#     p1, p2  = realised["pred_dir"].mean(), realised["actual_dir"].mean()
#     c_joint = (realised["pred_dir"] & realised["actual_dir"]).mean()
#     pt_p = 2 * (1 - stats.norm.cdf(abs((c_joint - p1*p2) /
#                                        np.sqrt(p1*p2*(1-p1)*(1-p2)/n))))
#     return hr, n, binom_p, pt_p

# def stratified_table(df):
#     rows = []
#     for lbl, (start, end) in REGIMES.items():
#         sub = df[(df["date"] >= start) & (df["date"] <= end)]
#         hr, n, bp, pt = dir_metrics(sub)
#         if np.isnan(hr):
#             continue
#         rows.append({"Regime": lbl, "Obs": n,
#                      "HitRate": hr, "Binom_p": bp, "PT_p": pt})
#     return pd.DataFrame(rows)

# # ─────────── 4) exhaustive search for k = 3 and 5  ────────────
# summary_rows  = []   # for final table
# verdict_votes = []   # collect live “Beat/Miss” signals

# for k in (3, 5):
#     if len(pool) < k:
#         print(f"Skipping k={k}: pool too small\n")
#         continue

#     combos_k = list(itertools.combinations(pool, k))

#     for win_lbl, start_date in WINDOWS.items():
#         best = {"combo": None, "hr": -np.inf, "n": None,
#                 "binom": np.nan, "pt": np.nan,
#                 "median_live": np.nan, "signal": "n/a",
#                 "df_full": None}

#         for combo in combos_k:
#             df_all = merged_oos(combo)
#             # window‑restricted view for choosing winner
#             df_eval = df_all if start_date is None else df_all[df_all["date"] >= start_date]
#             hr, n, bp, pt = dir_metrics(df_eval)
#             if hr > best["hr"]:
#                 # live month info
#                 unreleased = df_all[df_all["actual"].isna()].sort_values("date")
#                 if not unreleased.empty:
#                     last = unreleased.iloc[-1]
#                     signal = "Beat" if last["ens"] > last["median"] else "Miss"
#                     median_live = last["median"]
#                 else:
#                     signal, median_live = "n/a", np.nan

#                 best.update({"combo": combo, "hr": hr, "n": n,
#                              "binom": bp, "pt": pt,
#                              "median_live": median_live, "signal": signal,
#                              "df_full": df_all})

#         # ── print detailed results ───────────────────────────
#         print(f"[{win_lbl}]  Best ensemble  k={k}")
#         print(f"  Specs        : {best['combo']}")
#         print(f"  HitRate      : {best['hr']:.2%}  over {best['n']} months")
#         print(f"  Binom p‑val  : {best['binom']:.3f}   PT p‑val : {best['pt']:.3f}")
#         if best["signal"] != "n/a":
#             print(f"  Consensus med: {best['median_live']:.0f} k")
#         print(f"  Live signal  : {best['signal']}")

#         # stratified across FULL history (not limited to window)
#         st = stratified_table(best["df_full"])
#         if st.empty:
#             print("  Stratified: no realised data yet.\n")
#         else:
#             print("  Stratified performance:")
#             print(st.to_string(index=False, float_format=lambda x: f"{x:0.3f}"))
#             print()

#         # gather for summary / verdict
#         summary_rows.append({
#             "Window": win_lbl, "k": k,
#             "Specs": best["combo"],
#             "HitRate": best["hr"], "Obs": best["n"],
#             "Binom_p": best["binom"], "PT_p": best["pt"],
#             "LiveSignal": best["signal"]
#         })
#         verdict_votes.append(best["signal"])

# # ─────────── 5) summary table & final verdict  ────────────────
# summary_df = (pd.DataFrame(summary_rows)
#               .sort_values(["Window", "k"])
#               .reset_index(drop=True))
# pd.set_option("display.float_format", "{:.3f}".format)
# print("\n================  ENSEMBLE SUMMARY  ================\n")
# print(summary_df.to_string(index=False))

# # majority vote across the six “best” ensembles (ties → ‘No consensus’)
# valid_votes = [v for v in verdict_votes if v in ("Beat", "Miss")]
# if valid_votes:
#     beat_count = valid_votes.count("Beat")
#     miss_count = valid_votes.count("Miss")
#     if beat_count > miss_count:
#         verdict = "Beat"
#     elif miss_count > beat_count:
#         verdict = "Miss"
#     else:
#         verdict = "No consensus"
# else:
#     verdict = "No live signal available"

# print(f"\n>>> FINAL VERDICT (majority across ensembles):  {verdict}\n")

Total unique candidate specs selected: 13
Usable specs with Full‑panel OOS: 13

[FULL]  Best ensemble  k=3
  Specs        : ('ewma_w3_d0.75_mequal_weight', 'ewma_w6_d0.85_minverse_mse', 'soft_bma_w12_nu50')
  HitRate      : 60.63%  over 254 months
  Binom p‑val  : 0.001   PT p‑val : 0.001
  Consensus med: 110 k
  Live signal  : Miss
  Stratified performance:
                                     Regime  Obs  HitRate  Binom_p  PT_p
               2003‑06 to 2007‑12 (pre‑GFC)   43    0.581    0.360 0.258
                   2008‑01 to 2009‑12 (GFC)   24    0.667    0.152 0.202
2010‑01‑2014‑12 (early‑expansion, post GFC)   60    0.583    0.245 0.190
 2015‑01‑2019‑12 (late‑expansion, post GFC)   60    0.583    0.245 0.152
                 2020‑01 to 2022‑12 (COVID)   36    0.556    0.618 0.549
         2023‑01 to 2025-07-28 (post‑COVID)   31    0.742    0.011 0.037
                         Trailing 24‑months   24    0.708    0.064 0.112
                         Trailing 12‑months   12    0.5

In [120]:
# -------------------------------------------------------------
#  In‑sample robust‑ensemble search  (k = 3, 5)
#     – winner chosen on  FULL, T24M, T12M, T6M, T3M
#     – forecast direction decided by **majority vote**
#     – adds Accuracy × Consistency score  ACλ
# -------------------------------------------------------------
import itertools, math, warnings
from typing import List, Tuple
import numpy as np
import pandas as pd
from scipy import stats

# ─────────────────────  SETTINGS  ────────────────────────────
TODAY   = pd.Timestamp.today().normalize()
WINDOWS = {
    "FULL": None,
    "T24M": TODAY - pd.DateOffset(months=24),
    "T12M": TODAY - pd.DateOffset(months=12),
    "T6M" : TODAY - pd.DateOffset(months=6),
    "T3M" : TODAY - pd.DateOffset(months=3),
}

LAMBDA_AC = 1.0                 # weight on consistency term in AC‑score

# REGIMES (six structural blocks) *must* be in the session already
# e.g.  REGIMES = {
#   "2003‑06 to 2007‑12 (pre‑GFC)"         : ("2003‑06","2007‑12"),
#   "2008‑01 to 2009‑12 (GFC)"             : ("2008‑01","2009‑12"),
#   ...
# }

# ─────────── 1) candidate pool (unchanged)  ──────────────────
pool = set()
for model_id, panel_map in eval_tables.items():
    for panel in ["COVID", "Full"]:
        df = panel_map.get(panel)
        if df is None or df.empty:
            continue
        pool.add(df.loc[df["RMSE_smart"].idxmin(), "spec_id"])
        pool.add(df.loc[df["HitRate"].idxmax(),    "spec_id"])
        rob = df[(df["DM_p"] < .10) & (df["PT_p"] < .10)]
        if not rob.empty:
            pool.add(rob.loc[rob["RMSE_smart"].idxmin(), "spec_id"])

pool = sorted(pool)
print(f"Total unique candidate specs selected: {len(pool)}")

# ─────────── 2) collect FULL‑panel OOS frames  ───────────────
oos_full = {}
for mdl, panel_map in oos_maps.items():
    oos_full.update(panel_map.get("Full", {}))

pool = [s for s in pool if s in oos_full]
assert len(pool) >= 3, "Need ≥3 viable specs with Full‑panel OOS"
print(f"Usable specs with Full‑panel OOS: {len(pool)}\n")

# ─────────── 3) helpers  ─────────────────────────────────────
def merged_oos(combo: Tuple[str, ...]) -> pd.DataFrame:
    """Merge members; keep each smart forecast for majority vote."""
    dfs: List[pd.DataFrame] = []
    for i, sid in enumerate(combo):
        tmp = (oos_full[sid][["date", "smart", "median", "actual"]]
               .rename(columns={"smart":  f"smart_{i}",
                                "median": f"median_{i}",
                                "actual": f"actual_{i}"}))
        dfs.append(tmp)
    df = dfs[0]
    for d in dfs[1:]:
        df = df.merge(d, on="date")

    df["median"] = df["median_0"]
    df["actual"] = df["actual_0"]
    smart_cols   = [c for c in df.columns if c.startswith("smart_")]
    df["ens"]    = df[smart_cols].mean(axis=1)
    return df[["date", "ens", "median", "actual", *smart_cols]]

def dir_metrics(df: pd.DataFrame):
    """Hit‑Rate, Binom‑p, PT‑p with majority‑vote direction."""
    realised = df.dropna(subset=["actual"]).copy()
    if realised.empty:
        return np.nan, 0, np.nan, np.nan

    smart_cols = [c for c in realised.columns if c.startswith("smart_")]
    votes      = realised[smart_cols].gt(realised["median"], axis=0).sum(axis=1)
    realised["pred_dir"]   = (votes > len(smart_cols) / 2).astype(int)
    realised["actual_dir"] = (realised["actual"] > realised["median"]).astype(int)

    hits  = (realised["pred_dir"] == realised["actual_dir"]).sum()
    n     = len(realised)
    hr    = hits / n
    binom = stats.binomtest(hits, n, .5).pvalue
    p1, p2  = realised["pred_dir"].mean(), realised["actual_dir"].mean()
    c_joint = (realised["pred_dir"] & realised["actual_dir"]).mean()
    pt_p = 2*(1 - stats.norm.cdf(abs((c_joint - p1*p2) /
                                     math.sqrt(p1*p2*(1-p1)*(1-p2)/n))))
    return hr, n, binom, pt_p

def stratified_table(df: pd.DataFrame) -> pd.DataFrame:
    """Per‑regime diagnostics (six blocks)."""
    rows = []
    for lbl, (start, end) in REGIMES.items():
        sub = df[(df["date"] >= start) & (df["date"] <= end)]
        hr, n, bp, pt = dir_metrics(sub)
        if np.isnan(hr):
            continue
        rows.append({"Regime": lbl, "Obs": n,
                     "HitRate": hr, "Binom_p": bp, "PT_p": pt})
    return pd.DataFrame(rows)

def ac_score(overall_hr: float, block_hr: pd.Series, lam: float = LAMBDA_AC):
    """Compute Accuracy × Consistency score (lower = better)."""
    if block_hr.empty or len(block_hr) < 2:
        return np.nan
    return (1 - overall_hr) + lam * block_hr.std(ddof=1)

# ─────────── 4) exhaustive search for k = 3 and 5  ───────────
summary_rows, verdict_votes = [], []

for k in (3, 5):
    if len(pool) < k:
        print(f"Skipping k={k}: pool too small\n")
        continue

    for win_lbl, start_date in WINDOWS.items():
        best = {"combo": None, "hr": -np.inf}

        for combo in itertools.combinations(pool, k):
            df_all  = merged_oos(combo)
            df_eval = df_all if start_date is None else df_all[df_all["date"] >= start_date]
            hr, n, bp, pt = dir_metrics(df_eval)
            if hr > best["hr"]:
                best.update({"combo": combo, "hr": hr, "n": n,
                             "binom": bp, "pt": pt, "df_full": df_all})

        # ---------- live signal (majority vote on last unreleased) -----
        df_full = best["df_full"]
        unreleased = df_full[df_full["actual"].isna()].sort_values("date")
        if not unreleased.empty:
            last = unreleased.iloc[-1]
            smart_cols = [c for c in last.index if c.startswith("smart_")]
            votes      = sum(last[c] > last["median"] for c in smart_cols)
            best["signal"]      = "Beat" if votes > len(smart_cols)/2 else "Miss"
            best["median_live"] = last["median"]
        else:
            best["signal"], best["median_live"] = "n/a", np.nan

        # ---------- regime diagnostics & AC‑score ----------------------
        st = stratified_table(df_full)
        core_blocks = st[~st["Regime"].str.startswith("Trailing")]
        best["ac_score"] = ac_score(best["hr"], core_blocks["HitRate"])

        # ---------- pretty print --------------------------------------
        print(f"[{win_lbl}]  Best ensemble  k={k}")
        print(f"  Specs        : {best['combo']}")
        print(f"  HitRate      : {best['hr']:.2%}  over {best['n']} months")
        print(f"  Binom p‑val  : {best['binom']:.3f}   PT p‑val : {best['pt']:.3f}")
        print(f"  AC‑score(λ={LAMBDA_AC}) : {best['ac_score']:.4f}")
        if best["signal"] != "n/a":
            print(f"  Consensus med: {best['median_live']:.0f} k")
        print(f"  Live signal  : {best['signal']}")
        if not core_blocks.empty:
            print("  Stratified performance:")
            print(core_blocks.to_string(index=False,
                                        float_format=lambda x: f"{x:0.3f}"))
        print()

        # gather rows for summary & verdict
        summary_rows.append({
            "Window": win_lbl, "k": k,
            "Specs": best["combo"],
            "HitRate": best["hr"], "Obs": best["n"],
            "Binom_p": best["binom"], "PT_p": best["pt"],
            f"ACλ={LAMBDA_AC}": best["ac_score"],
            "LiveSignal": best["signal"]
        })
        verdict_votes.append(best["signal"])

# ─────────── 5) summary table & majority verdict  ────────────
summary_df = (pd.DataFrame(summary_rows)
              .sort_values(["Window", "k"])
              .reset_index(drop=True))
pd.set_option("display.float_format", "{:.3f}".format)

print("\n================  ENSEMBLE SUMMARY  ================\n")
print(summary_df.to_string(index=False))

valid = [v for v in verdict_votes if v in ("Beat", "Miss")]
if valid:
    beat_cnt = valid.count("Beat")
    miss_cnt = valid.count("Miss")
    verdict  = "Beat" if beat_cnt > miss_cnt else (
               "Miss" if miss_cnt > beat_cnt else "No consensus")
else:
    verdict = "No live signal available"

print(f"\n>>> FINAL VERDICT (majority across ensembles):  {verdict}\n")


Total unique candidate specs selected: 13
Usable specs with Full‑panel OOS: 13

[FULL]  Best ensemble  k=3
  Specs        : ('ewma_w12_d0.95_minverse_mse', 'mwu_w3_eta0.050', 'soft_bma_w12_nu50')
  HitRate      : 59.45%  over 254 months
  Binom p‑val  : 0.003   PT p‑val : 0.003
  AC‑score(λ=1.0) : 0.4598
  Consensus med: 110 k
  Live signal  : Miss
  Stratified performance:
                                     Regime  Obs  HitRate  Binom_p  PT_p
               2003‑06 to 2007‑12 (pre‑GFC)   43    0.558    0.542 0.432
                   2008‑01 to 2009‑12 (GFC)   24    0.667    0.152 0.157
2010‑01‑2014‑12 (early‑expansion, post GFC)   60    0.550    0.519 0.466
 2015‑01‑2019‑12 (late‑expansion, post GFC)   60    0.600    0.155 0.108
                 2020‑01 to 2022‑12 (COVID)   36    0.583    0.405 0.393
         2023‑01 to 2025-07-28 (post‑COVID)   31    0.677    0.071 0.227

[T24M]  Best ensemble  k=3
  Specs        : ('ewma_w3_d0.75_mequal_weight', 'ewma_w6_d0.85_minverse_mse', 'soft

In [ ]:
import itertools, math
from collections import defaultdict
from typing import List, Tuple, Dict, Any

import numpy as np
import pandas as pd
from scipy import stats
from tqdm.auto import tqdm

# ---------------- configuration -----------------------------
WINDOWS = {6: "T6", 12: "T12", 24: "T24"}     # trailing windows
K_SET   = (3, 5)                     # ensemble sizes
ALPHA   = 0.10                       # p‑value threshold for robust winner
# ------------------------------------------------------------

# ---------- helper functions (unchanged except tqdm) --------
def _spec_metrics(oos: pd.DataFrame) -> Dict[str, Any]:
    df = oos.dropna(subset=["actual"]).copy()
    if df.empty:
        return {"obs": 0, "rmse": np.nan, "hr": np.nan,
                "dm_p": np.nan, "pt_p": np.nan}

    df["smart_err"]  = df["smart"]  - df["actual"]
    df["median_err"] = df["median"] - df["actual"]
    df["actual_dir"] = (df["actual"] > df["median"]).astype(int)

    obs  = len(df)
    rmse = np.sqrt((df["smart_err"] ** 2).mean())
    hr   = (df["pred_dir"] == df["actual_dir"]).mean()

    diff = df["smart_err"] ** 2 - df["median_err"] ** 2
    dm_p = (1.0 if diff.std(ddof=1) == 0 else
            2 * (1 - stats.norm.cdf(abs(diff.mean() /
                                         diff.std(ddof=1) *
                                         math.sqrt(obs)))))

    p1, p2 = df["pred_dir"].mean(), df["actual_dir"].mean()
    denom  = p1 * p2 * (1 - p1) * (1 - p2)
    pt_p   = (1.0 if denom == 0 else
              2 * (1 - stats.norm.cdf(abs(((df["pred_dir"] &
                                             df["actual_dir"]).mean() -
                                             p1 * p2) /
                                           math.sqrt(denom / obs)))))

    return {"obs": obs, "rmse": rmse, "hr": hr, "dm_p": dm_p, "pt_p": pt_p}


def _candidate_pool(stats_df: pd.DataFrame) -> List[str]:
    chosen = set()
    for (mdl, pnl), grp in stats_df.groupby(["model", "panel"]):
        grp = grp[grp["obs"] > 0]
        if grp.empty:
            continue
        chosen.add(grp.loc[grp["rmse"].idxmin(), "spec_id"])
        chosen.add(grp.loc[grp["hr"].idxmax(),   "spec_id"])
        robust = grp[(grp["dm_p"] < ALPHA) & (grp["pt_p"] < ALPHA)]
        if not robust.empty:
            chosen.add(robust.loc[robust["rmse"].idxmin(), "spec_id"])
    return sorted(chosen)


_combo_cache: Dict[Tuple[str, ...], pd.DataFrame] = {}
def _merged_oos_cached(combo: Tuple[str, ...]):
    if combo not in _combo_cache:
        _combo_cache[combo] = merged_oos(combo)
    return _combo_cache[combo]


def _combo_hitrate(combo: Tuple[str, ...], t_cut: pd.Timestamp,
                   window: int) -> Tuple[float, int]:
    df = _merged_oos_cached(combo)
    sub = df[(df["date"] >= t_cut - pd.DateOffset(months=window)) &
             (df["date"] <  t_cut) & df["actual"].notna()].copy()
    if sub.empty:
        return np.nan, 0
    sub["actual_dir"] = (sub["actual"] > sub["median"]).astype(int)
    hr = (sub["ens"] > sub["median"]).eq(sub["actual_dir"]).mean()
    return hr, len(sub)


def _choose_best_combo(pool: List[str], t_cut: pd.Timestamp,
                       window: int) -> Tuple[Tuple[str, ...], float]:
    best_combo, best_hr, best_rmse = None, -1.0, np.inf
    for k in K_SET:
        if len(pool) < k:
            continue
        for combo in itertools.combinations(pool, k):
            hr, _ = _combo_hitrate(combo, t_cut, window)
            if math.isnan(hr):
                continue
            if hr > best_hr:
                best_combo, best_hr = combo, hr
                best_rmse = _window_rmse(combo, t_cut, window)
            elif hr == best_hr:
                rmse = _window_rmse(combo, t_cut, window)
                if rmse < best_rmse or (rmse == best_rmse and combo < best_combo):
                    best_combo, best_rmse = combo, rmse
    return best_combo, best_hr


def _window_rmse(combo: Tuple[str, ...], t_cut: pd.Timestamp, window: int):
    df = _merged_oos_cached(combo)
    sub = df[(df["date"] >= t_cut - pd.DateOffset(months=window)) &
             (df["date"] <  t_cut) & df["actual"].notna()]
    return (np.inf if sub.empty else
            math.sqrt(((sub["ens"] - sub["actual"]) ** 2).mean()))


def _ensemble_direction(combo: Tuple[str, ...], t_date: pd.Timestamp) -> int:
    df = _merged_oos_cached(combo)
    row = df[df["date"] == t_date].iloc[0]
    return int(row["ens"] > row["median"])


def _actual_direction(spec_any: pd.DataFrame, t_date: pd.Timestamp) -> int:
    row = spec_any[spec_any["date"] == t_date].iloc[0]
    return int(row["actual"] > row["median"])


def stratified_table(df_preds: pd.DataFrame) -> pd.DataFrame:
    """Directional diagnostics by regime; columns coerced to int."""
    df = df_preds.copy()
    df["pred_dir"]   = df["pred_dir"].astype(int)
    df["actual_dir"] = df["actual_dir"].astype(int)

    rows = []
    for lbl, (start, end) in REGIMES.items():
        sub = df[(df["date"] >= start) & (df["date"] <= end)]
        if sub.empty:
            continue

        hits = (sub["pred_dir"] == sub["actual_dir"]).sum()
        n    = len(sub)
        hr   = hits / n
        binom_p = stats.binomtest(hits, n, .5).pvalue

        p1, p2 = sub["pred_dir"].mean(), sub["actual_dir"].mean()
        denom  = p1 * p2 * (1 - p1) * (1 - p2)
        if denom == 0:
            pt_p = 1.0
        else:
            joint = (sub["pred_dir"].astype(int) & sub["actual_dir"].astype(int)).mean()
            pt_stat = (joint - p1 * p2) / math.sqrt(denom / n)
            pt_p = 2 * (1 - stats.norm.cdf(abs(pt_stat)))

        rows.append({"Regime": lbl, "Obs": n,
                     "HitRate": hr, "Binom_p": binom_p, "PT_p": pt_p})

    return pd.DataFrame(rows)
# ------------------------------------------------------------
# 2.  Timeline & warm‑up discovery
# ------------------------------------------------------------
all_dates = sorted({pd.to_datetime(d)
                    for mdl in oos_maps.values()
                    for pnl in mdl.values()
                    for o in pnl.values()
                    for d in o["date"].unique()})

warmup_idx = 0
with tqdm(total=len(all_dates), desc="Finding warm‑up start") as pbar:
    while warmup_idx < len(all_dates):
        t0 = all_dates[warmup_idx]
        ok = True
        for mdl in oos_maps.values():
            for pnl in mdl.values():
                for oos in pnl.values():
                    realised = oos[(oos["date"] < t0) & oos["actual"].notna()]
                    if len(realised) < 24:
                        ok = False
                        break
                if not ok:
                    break
            if not ok:
                break
        if ok:
            break
        warmup_idx += 1
        pbar.update(1)

print(f"Warm‑up starts at {all_dates[warmup_idx].date()}")

# ------------------------------------------------------------
# 3.  Rolling evaluation with progress bar
# ------------------------------------------------------------
records = {tag: [] for tag in WINDOWS.values()}
dates_iter = tqdm(all_dates[warmup_idx:], desc="Rolling evaluation", unit="month")

# pick one oos frame just to fetch actuals quickly
sample_oos_any = next(iter(
    oos_maps[next(iter(oos_maps))]["Full"].values()
))

for t in dates_iter:
    # --- per‑spec stats -------------------------------------
    rows_stats = []
    for model_id, panel_map in oos_maps.items():
        for panel_name, spec_dict in panel_map.items():
            for spec_id, oos in spec_dict.items():
                metrics = _spec_metrics(oos[oos["date"] < t])
                metrics.update({"model": model_id,
                                "panel": panel_name,
                                "spec_id": spec_id})
                rows_stats.append(metrics)
    stats_df = pd.DataFrame(rows_stats)
    if stats_df["obs"].max() < 12:
        continue

    # --- candidate pool -------------------------------------
    pool_t = _candidate_pool(stats_df)
    if len(pool_t) < 3:
        continue

    # --- choose best combos for each window -----------------
    best_combo_dict = {}
    for W in WINDOWS:
        combo, hr = _choose_best_combo(pool_t, t, W)
        if combo is not None:
            best_combo_dict[WINDOWS[W]] = combo

    if not best_combo_dict:
        continue

    actual_known = not math.isnan(
        sample_oos_any[sample_oos_any["date"] == t]["actual"].iloc[0]
    )

    # --- live predictions & logging -------------------------
    for tag, combo in best_combo_dict.items():
        pred_dir = _ensemble_direction(combo, t)
        rec = {"date": t, "pred_dir": pred_dir}
        if actual_known:
            rec["actual_dir"] = _actual_direction(sample_oos_any, t)
            rec["hit"] = int(rec["pred_dir"] == rec["actual_dir"])
        records[tag].append(rec)

# -------------------- build overall summary -----------------------
summary_rows, strat_tables = [], {}
for tag, recs in records.items():
    df_preds = (pd.DataFrame(recs)
                  .dropna(subset=["hit"])
                  .assign(pred_dir=lambda d: d["pred_dir"].astype(int),
                          actual_dir=lambda d: d["actual_dir"].astype(int)))

    summary_rows.append({
        "Method": tag,
        "Obs": len(df_preds),
        "HitRate": df_preds["hit"].mean() if not df_preds.empty else np.nan
    })

    if not df_preds.empty:
        strat_tables[tag] = stratified_table(df_preds)

print("\n=== Dynamic robust‑ensemble back‑test summary ===")
print(pd.DataFrame(summary_rows)
        .to_string(index=False, float_format=lambda x: f"{x:0.3f}"))

for tag, tbl in strat_tables.items():
    print(f"\n--- Stratified diagnostics for {tag} ---")
    print(tbl.to_string(index=False, float_format=lambda x: f"{x:0.3f}"))


Finding warm‑up start:   0%|          | 0/264 [00:00<?, ?it/s]

Warm‑up starts at 2006-06-02


Rolling evaluation:   0%|          | 0/231 [00:00<?, ?month/s]


=== Dynamic robust‑ensemble back‑test summary ===
Method  Obs  HitRate
    T6  230    0.535
   T12  230    0.578
   T24  230    0.570

--- Stratified diagnostics for T6 ---
                                     Regime  Obs  HitRate  Binom_p  PT_p
               2003‑06 to 2007‑12 (pre‑GFC)   19    0.421    0.648 0.515
                   2008‑01 to 2009‑12 (GFC)   24    0.500    1.000 0.744
2010‑01‑2014‑12 (early‑expansion, post GFC)   60    0.517    0.897 0.726
 2015‑01‑2019‑12 (late‑expansion, post GFC)   60    0.483    0.897 0.791
                 2020‑01 to 2022‑12 (COVID)   36    0.611    0.243 0.196
         2023‑01 to 2025-07-28 (post‑COVID)   31    0.677    0.071 0.360
                         Trailing 24‑months   24    0.625    0.307 0.525
                         Trailing 12‑months   12    0.583    0.774 0.679

--- Stratified diagnostics for T12 ---
                                     Regime  Obs  HitRate  Binom_p  PT_p
               2003‑06 to 2007‑12 (pre‑GFC)   19    0.52

In [ ]:
# ==============================================================
#  Dynamic majority‑vote ensembles   (2017‑01 → present)
#    • trailing‑window winners  T3 / T6 / T12 / T24
#    • Best‑to‑Date (BTD) single‑spec benchmark
#    • Accuracy × Consistency score per method (λ = 1.0)
#    • Stratified diagnostics: 2017‑19 / 2020‑22 / 2023‑‑
#    • Evolution plot of winner hit‑rate (five lines)
# ==============================================================

import itertools, math, warnings, matplotlib.pyplot as plt
from collections import defaultdict
from typing import List, Tuple, Dict, Any

import numpy as np
import pandas as pd
from scipy import stats
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")

# ---------- configuration ------------------------------------
EVAL_START = pd.Timestamp("2017-01-01")            # first forecast recorded
WINDOWS    = {3: "T3", 6: "T6", 12: "T12", 24: "T24"}
K_SET      = (3, 5)                                # ensemble sizes
ALPHA      = 0.10                                  # robust‑winner p‑value
LAMBDA_AC  = 1.0                                   # weight in A×C score

REGIMES = {
    "Pre‑COVID (2017‑19)": (pd.Timestamp("2017-01-01"),
                            pd.Timestamp("2019-12-31")),
    "COVID (2020‑22)"    : (pd.Timestamp("2020-01-01"),
                            pd.Timestamp("2022-12-31")),
    "Post‑COVID (2023‑‑)": (pd.Timestamp("2023-01-01"),
                            pd.Timestamp.today()),
}

# ---------- helper functions ---------------------------------
def _spec_metrics(oos: pd.DataFrame) -> Dict[str, Any]:
    df = oos.dropna(subset=["actual"]).copy()
    if df.empty:
        return dict(obs=0, rmse=np.nan, hr=np.nan, dm_p=np.nan, pt_p=np.nan)

    df["smart_err"]  = df["smart"]  - df["actual"]
    df["median_err"] = df["median"] - df["actual"]
    df["pred_dir"]   = (df["smart"]  > df["median"]).astype(int)
    df["actual_dir"] = (df["actual"] > df["median"]).astype(int)

    obs  = len(df)
    rmse = np.sqrt((df["smart_err"]**2).mean())
    hr   = (df["pred_dir"] == df["actual_dir"]).mean()

    diff = df["smart_err"]**2 - df["median_err"]**2
    dm_p = 1.0 if diff.std(ddof=1) == 0 else \
           2*(1-stats.norm.cdf(abs(diff.mean()/diff.std(ddof=1)*np.sqrt(obs))))

    p1, p2 = df["pred_dir"].mean(), df["actual_dir"].mean()
    denom  = p1*p2*(1-p1)*(1-p2)
    pt_p   = 1.0 if denom == 0 else \
             2*(1-stats.norm.cdf(abs(((df["pred_dir"] & df["actual_dir"]).mean()-p1*p2) /
                                     np.sqrt(denom/obs))))

    return dict(obs=obs, rmse=rmse, hr=hr, dm_p=dm_p, pt_p=pt_p)


def _candidate_pool(stats_df: pd.DataFrame) -> List[str]:
    chosen = set()
    for (_, _), grp in stats_df.groupby(["model", "panel"]):
        grp = grp[grp["obs"] > 0]
        if grp.empty:
            continue
        chosen.add(grp.loc[grp["rmse"].idxmin(), "spec_id"])   # lowest RMSE
        chosen.add(grp.loc[grp["hr"].idxmax(),   "spec_id"])   # highest HR
        rob = grp[(grp["dm_p"] < ALPHA) & (grp["pt_p"] < ALPHA)]
        if not rob.empty:
            chosen.add(rob.loc[rob["rmse"].idxmin(), "spec_id"])
    return sorted(chosen)


_combo_cache: Dict[Tuple[str, ...], pd.DataFrame] = {}
def _merged_oos_cached(combo: Tuple[str, ...]) -> pd.DataFrame:
    if combo not in _combo_cache:
        _combo_cache[combo] = merged_oos(combo)     # requires earlier cells
    return _combo_cache[combo]


def _mv_hitrate(df: pd.DataFrame) -> float:
    if df.empty:
        return np.nan
    smart_cols = [c for c in df.columns if c.startswith("smart_")]
    votes      = (df[smart_cols].gt(df["median"], axis=0)).sum(axis=1)
    pred_dir   = (votes > len(smart_cols)/2).astype(int)
    actual_dir = (df["actual"] > df["median"]).astype(int)
    return (pred_dir == actual_dir).mean()


def _combo_hitrate(combo, t_cut, window):
    df = _merged_oos_cached(combo)
    sub = df[(df["date"] >= t_cut - pd.DateOffset(months=window)) &
             (df["date"] <  t_cut) & df["actual"].notna()]
    return _mv_hitrate(sub), len(sub)


def _window_rmse(combo, t_cut, window):
    df = _merged_oos_cached(combo)
    sub = df[(df["date"] >= t_cut - pd.DateOffset(months=window)) &
             (df["date"] <  t_cut) & df["actual"].notna()]
    return (np.inf if sub.empty else np.sqrt(((sub["ens"]-sub["actual"])**2).mean()))


def _choose_best_combo(pool, t_cut, window):
    best_combo, best_hr, best_rmse = None, -1.0, np.inf
    for k in K_SET:
        if len(pool) < k:
            continue
        for combo in itertools.combinations(pool, k):
            hr, _ = _combo_hitrate(combo, t_cut, window)
            if math.isnan(hr):
                continue
            if hr > best_hr:
                best_combo, best_hr = combo, hr
                best_rmse = _window_rmse(combo, t_cut, window)
            elif hr == best_hr:
                rmse = _window_rmse(combo, t_cut, window)
                if rmse < best_rmse or (rmse == best_rmse and combo < best_combo):
                    best_combo, best_rmse = combo, rmse
    return best_combo, best_hr


def _ensemble_direction_mv(combo, t_date):
    df = _merged_oos_cached(combo)
    row = df[df["date"] == t_date].iloc[0]
    smart_cols = [c for c in row.index if c.startswith("smart_")]
    votes = sum(row[c] > row["median"] for c in smart_cols)
    return int(votes > len(smart_cols)/2)


def _actual_direction(any_spec_df, t_date):
    row = any_spec_df[any_spec_df["date"] == t_date].iloc[0]
    return int(row["actual"] > row["median"])


def _stratified(df_preds: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for lbl, (s, e) in REGIMES.items():
        sub = df_preds[(df_preds["date"] >= s) & (df_preds["date"] <= e)]
        if sub.empty:
            continue
        hits = sub["hit"].mean()
        n    = len(sub)
        binom = stats.binomtest(sub["hit"].sum(), n, .5).pvalue
        p1, p2 = sub["pred_dir"].mean(), sub["actual_dir"].mean()
        denom  = p1*p2*(1-p1)*(1-p2)
        pt     = 1.0 if denom == 0 else \
                 2*(1-stats.norm.cdf(abs(((sub["pred_dir"] & sub["actual_dir"]).mean()-p1*p2) /
                                         math.sqrt(denom/n))))
        rows.append(dict(Regime=lbl, Obs=n, HitRate=hits, Binom_p=binom, PT_p=pt))
    return pd.DataFrame(rows)


def _ac_score(overall_hr, reg_hits, lam=LAMBDA_AC):
    if len(reg_hits) < 2 or math.isnan(overall_hr):
        return np.nan
    return (1 - overall_hr) + lam*np.std(reg_hits, ddof=1)

# -------------------------------------------------------------
# 1. timeline + warm‑up discovery (≥24 realised obs)
# -------------------------------------------------------------
all_dates = sorted({pd.to_datetime(d)
                    for mdl in oos_maps.values()
                    for pnl in mdl.values()
                    for o in pnl.values()
                    for d in o["date"].unique()})

warmup_idx = 0
with tqdm(total=len(all_dates), desc="Finding warm‑up start") as bar:
    while warmup_idx < len(all_dates) and all_dates[warmup_idx] < EVAL_START:
        warmup_idx += 1
        bar.update(1)

# -------------------------------------------------------------
# 2. rolling evaluation
# -------------------------------------------------------------
records   = {tag: [] for tag in WINDOWS.values()}
records["BTD"] = []
sel_hr_ts = {tag: [] for tag in list(WINDOWS.values()) + ["BTD"]}

# helpers for BTD
cum_hits: Dict[str, int] = defaultdict(int)
cum_obs : Dict[str, int] = defaultdict(int)

dates_iter = tqdm(all_dates[warmup_idx:], desc="Rolling evaluation", unit="month")
sample_oos_any = next(iter(oos_maps[next(iter(oos_maps))]["Full"].values()))

for t in dates_iter:
    if t < EVAL_START:
        continue

    # --- per‑spec metrics through t‑1 -----------------------
    rows_stats = []
    for m_id, pnl_map in oos_maps.items():
        for p_name, spec_dict in pnl_map.items():
            for sid, oos in spec_dict.items():
                met = _spec_metrics(oos[oos["date"] < t])
                met.update(model=m_id, panel=p_name, spec_id=sid)
                rows_stats.append(met)
    stats_df = pd.DataFrame(rows_stats)
    if stats_df["obs"].max() < 12:
        continue

    # --- candidate pool -------------------------------------
    pool_t = _candidate_pool(stats_df)
    if len(pool_t) < 3:
        continue

    # --- choose MV winners per window -----------------------
    best_combo: Dict[str, Tuple[str, ...]] = {}
    win_hr     : Dict[str, float]          = {}

    for W in WINDOWS:
        combo, hr = _choose_best_combo(pool_t, t, W)
        if combo:
            tag = WINDOWS[W]
            best_combo[tag] = combo
            win_hr[tag]     = hr

    # --- BTD single‑spec ------------------------------------
    for row in stats_df.itertuples():
        cum_hits[row.spec_id] = int(row.hr * row.obs)   # store total hits to date
        cum_obs [row.spec_id] = row.obs                 # store total obs to date
    if cum_obs:
        best_sid = max(cum_obs, key=lambda s: cum_hits[s] / cum_obs[s])
        best_combo["BTD"] = (best_sid,)
        win_hr["BTD"]     = cum_hits[best_sid] / cum_obs[best_sid]

    if not best_combo:
        continue

    # --- actual known? --------------------------------------
    actual_known = pd.notna(sample_oos_any.loc[sample_oos_any["date"] == t, "actual"].iloc[0])

    # --- record predictions & hit‑rate evolution ------------
    for tag, combo in best_combo.items():
        pred_dir = _ensemble_direction_mv(combo, t)  # works for 1- or many‑member
        rec = dict(date=t, pred_dir=pred_dir, sel_hr=win_hr[tag])
        if actual_known:
            rec["actual_dir"] = _actual_direction(sample_oos_any, t)
            rec["hit"]        = int(rec["pred_dir"] == rec["actual_dir"])
        records[tag].append(rec)
        sel_hr_ts[tag].append((t, win_hr[tag]))

# -------------------------------------------------------------
# 3. summaries, stratified diagnostics, AC score
# -------------------------------------------------------------
summary_rows, strat_tables = [], {}

for tag, recs in records.items():
    dfp = (pd.DataFrame(recs)
             .dropna(subset=["hit"])
             .assign(pred_dir=lambda d: d["pred_dir"].astype(int),
                     actual_dir=lambda d: d["actual_dir"].astype(int)))

    ov_hr = dfp["hit"].mean() if not dfp.empty else np.nan
    strat = _stratified(dfp) if not dfp.empty else pd.DataFrame()
    ac    = np.nan
    if not strat.empty:
        ac = _ac_score(ov_hr, strat["HitRate"].tolist(), LAMBDA_AC)
        strat_tables[tag] = strat

    summary_rows.append(dict(Method=tag, Obs=len(dfp), HitRate=ov_hr,
                             **{f"ACλ={LAMBDA_AC}": ac}))

# -------------------------------------------------------------
# 4. plotting winner‑spec hit‑rate evolution
# -------------------------------------------------------------
plt.figure(figsize=(10, 5))
for tag, pts in sel_hr_ts.items():
    if not pts:
        continue
    pts_sorted = sorted(pts, key=lambda x: x[0])
    dates, hrs = zip(*pts_sorted)
    plt.plot(dates, hrs, label=tag)
plt.axhline(0.5, ls="--", lw=0.8, color="grey")
plt.ylabel("Hit‑rate of winning ensemble/spec")
plt.title("Evolution of selection‑window hit‑rate (Jan‑2017 → present)")
plt.legend()
plt.tight_layout()
plt.show()

# -------------------------------------------------------------
# 5. console output
# -------------------------------------------------------------
pd.set_option("display.float_format", "{:.3f}".format)

print("\n=== Back‑test summary (2017‑‑) ===")
print(pd.DataFrame(summary_rows).to_string(index=False))

for tag, tbl in strat_tables.items():
    print(f"\n--- Stratified diagnostics for {tag} ---")
    print(tbl.to_string(index=False,
                        float_format=lambda x: f"{x:0.3f}"))
    ac_val = next(r[f"ACλ={LAMBDA_AC}"] for r in summary_rows
                  if r["Method"] == tag)
    print(f"Accuracy × Consistency (λ = {LAMBDA_AC}):  {ac_val:0.3f}\n")

print("\n==============  Summary Table  ==============")
print(pd.DataFrame(summary_rows)
        .set_index("Method")
        .to_string(float_format=lambda x: f"{x:0.3f}"))


Finding warm‑up start:   0%|          | 0/264 [00:00<?, ?it/s]

Rolling evaluation:   0%|          | 0/104 [00:00<?, ?month/s]

In [ ]:
oos_maps

## 2 Distributional forecasting

We deploy the following distributional forecasting methods to quantify forecast uncertainty
1. Student-t fitted to past errors
2. t-GARCH: Rolling-36 GARCH(1,1) with Student-t innovations 
3. Gaussian Mixture Models
4. Bayesian Model Averaging
5. Kalman filter (TODO)

### 2.1 Student-t error bands

In [43]:
# settings
ROLL_WIN   = 36
MIN_TRAIN  = 36
LEVELS     = np.array([.50,.60,.70,.80,.90,.95])
BETA_BASE  = 0.0
BETA_CRIS  = 0.80
PCTL_THRES = 0.95
LAMBDA     = 1.0
PANELS     = {"COVID": df, "Full": df_full}

warnings.filterwarnings("ignore")
np.seterr(all="ignore")

ci_half = lambda lvl, df_, sig: st.t.ppf(1-(1-lvl)/2, df=df_)*sig

# ---------------------------------------------------------------------
# Candidate spec grid  (name, window_type, use_mu_shift)
# CrisisAdj toggled later so we get 9 total variants
# ---------------------------------------------------------------------
SPEC_BASE = [
    ("Exp_noShift" , "expanding", False),
    ("Roll36_noShift", "rolling", False),
    ("Roll36_plusMu" , "rolling", True ),
]

# Build a reverse‑lookup to reconstruct settings from a tag --------
spec_params = {}
for nm, wt, um in SPEC_BASE:
    for crisis in (False, True):
        tag = f"{nm}_{'Yes' if crisis else 'No'}"
        spec_params[tag] = dict(name=nm, wtype=wt,
                                use_mu=um, crisis=crisis)

# =====================================================================
# BACKTEST  (both panels, all tags)
# =====================================================================
cov_rows, gap_rows = [], []

for p_label, panel in PANELS.items():
    # per‑release aggregates ------------------------------------------
    rel = (panel.groupby("release_date")
                 .agg(median_fc=("median_forecast","median"),
                      actual    =("actual","first"),
                      spread    =("forecast","std"))
                 .reset_index()
                 .sort_values("release_date"))
    rel["err"] = rel["median_fc"] - rel["actual"]
    eval_df = rel[rel["actual"].notna()].reset_index(drop=True)

    for nm, wtype, use_mu in SPEC_BASE:
        for crisis in (False, True):
            tag   = f"{nm}_{'Yes' if crisis else 'No'}"
            hits  = defaultdict(int);  totN = 0

            for i in tqdm(range(MIN_TRAIN, len(eval_df)),
                          desc=f"{p_label} | {tag}", leave=False):

                # training window -----------------------------------
                if wtype == "expanding":
                    errs = eval_df.loc[:i-1, "err"].values
                    sprd = eval_df.loc[:i-1, "spread"].values
                else:
                    if i < ROLL_WIN:  continue
                    errs = eval_df.loc[i-ROLL_WIN:i-1, "err"].values
                    sprd = eval_df.loc[i-ROLL_WIN:i-1, "spread"].values

                nu, mu, sig = st.t.fit(errs)

                pt      = eval_df.at[i, "median_fc"]
                act     = eval_df.at[i, "actual"]
                sp_t    = eval_df.at[i, "spread"]

                # crisis multiplier -------------------------------
                if crisis:
                    med_sp = np.median(sprd)
                    pct    = (sprd < sp_t).mean()
                    beta   = BETA_BASE if pct < PCTL_THRES else BETA_CRIS
                    mult   = (sp_t/med_sp)**beta
                else:
                    mult   = 1.0

                centre = pt + (mu if use_mu else 0.0)

                for L in LEVELS:
                    h = ci_half(L, nu, sig)*mult
                    if centre-h <= act <= centre+h:
                        hits[L] += 1
                totN += 1

            if totN == 0:  continue
            emp = np.array([hits[L]/totN for L in LEVELS])
            for L, e in zip(LEVELS, emp):
                cov_rows.append(dict(Panel=p_label, Spec=tag,
                                     Nominal=L, Empirical=e))
            gap_rows.append(dict(Panel=p_label, Spec=tag,
                                 AvgAbsGap=float(np.abs(emp-LEVELS).mean())))

# ------------ tidy back‑test tables ---------------------------
cov_tbl = (pd.DataFrame(cov_rows)
             .pivot_table(index=["Panel","Spec"],
                          columns="Nominal", values="Empirical")
             .sort_index())

gap_tbl = (pd.DataFrame(gap_rows)
             .set_index(["Panel","Spec"])
             .sort_index())

print("\n=== Back‑test coverage (Student‑t) ===")
print(cov_tbl)
print("\nMean‑absolute coverage gap (lower = better):")
print(gap_tbl)

# =====================================================================
# pick best spec
# =====================================================================
best_row = (gap_tbl.xs("Full")
                     .sort_values("AvgAbsGap")
                     .iloc[0])
best_tag = best_row.name         # MultiIndex -> tag str
best_cfg = spec_params[best_tag]

print("\n>>> Best spec on FULL panel <<<")
print(f"  Tag          : {best_tag}")
print(f"  Window type  : {best_cfg['wtype']}")
print(f"  μ‑shift      : {best_cfg['use_mu']}")
print(f"  CrisisAdj    : {best_cfg['crisis']}")
print(f"  AvgAbsGap    : {best_row['AvgAbsGap']:.4f}")

# =====================================================================
# helper to compute half‑width multiplier  (crisis / no‑crisis)
# =====================================================================
def crisis_mult(sp_hist, sp_t, crisis_flag):
    if not crisis_flag:
        return 1.0
    med_sp = np.median(sp_hist)
    pct    = (sp_hist < sp_t).mean()
    beta   = BETA_BASE if pct < PCTL_THRES else BETA_CRIS
    return (sp_t / med_sp) ** beta

# =====================================================================
# STRATIFIED (best spec)
# =====================================================================
print("\n=== Stratified 4‑block results (best spec) ===")
cov_rows, gap_rows = [], []

for p_label, panel in PANELS.items():
    rel = (panel.groupby("release_date")
                 .agg(median_fc=("median_forecast", "median"),
                      actual    =("actual",          "first"),
                      spread    =("forecast",        "std"))
                 .reset_index()
                 .sort_values("release_date"))
    rel = rel[rel["actual"].notna()].reset_index(drop=True)
    rel["err"] = rel["median_fc"] - rel["actual"]

    idx_eval = np.arange(MIN_TRAIN, len(rel))
    blocks   = np.array_split(idx_eval, 4)

    g_hits = defaultdict(int); g_tot = 0

    for b_no, idx in enumerate(blocks, 1):
        hits = defaultdict(int); tot = 0
        st_dt = rel.at[idx[0], "release_date"].date()
        en_dt = rel.at[idx[-1], "release_date"].date()

        for j in tqdm(idx, desc=f"{p_label} | Blk{b_no}", leave=False):
            # training window per spec ---------------------------
            if best_cfg['wtype'] == "expanding":
                err_h = rel.loc[:j-1, "err"].values
                sp_h  = rel.loc[:j-1, "spread"].values
            else:                                           # rolling‑36
                if j < ROLL_WIN:  continue
                err_h = rel.loc[j-ROLL_WIN:j-1, "err"].values
                sp_h  = rel.loc[j-ROLL_WIN:j-1, "spread"].values

            nu, mu, sig = st.t.fit(err_h)

            centre  = rel.at[j, "median_fc"] + (mu if best_cfg['use_mu'] else 0.0)
            actual  = rel.at[j, "actual"]
            sp_t    = rel.at[j, "spread"]
            mult    = crisis_mult(sp_h, sp_t, best_cfg['crisis'])

            for L in LEVELS:
                h = ci_half(L, nu, sig) * mult
                hit = int(centre - h <= actual <= centre + h)
                hits[L]   += hit
                g_hits[L] += hit
            tot   += 1
            g_tot += 1

        emp = np.array([hits[L] / tot for L in LEVELS])
        cov_rows.extend([dict(Panel=p_label, Block=b_no,
                              Start=st_dt, End=en_dt,
                              Nominal=L, Empirical=e)
                         for L, e in zip(LEVELS, emp)])
        gap_rows.append(dict(Panel=p_label, Block=b_no,
                             Start=st_dt, End=en_dt,
                             AvgAbsGap=float(np.abs(emp - LEVELS).mean())))

    # global “All” row
    emp_all = np.array([g_hits[L] / g_tot for L in LEVELS])
    cov_rows.extend([dict(Panel=p_label, Block="All",
                          Start=rel.iloc[0]["release_date"].date(),
                          End  =rel.iloc[-1]["release_date"].date(),
                          Nominal=L, Empirical=e)
                     for L, e in zip(LEVELS, emp_all)])
    gap_rows.append(dict(Panel=p_label, Block="All",
                         Start=rel.iloc[0]["release_date"].date(),
                         End  =rel.iloc[-1]["release_date"].date(),
                         AvgAbsGap=float(np.abs(emp_all - LEVELS).mean())))

# ---------- tidy + print ----------
cov_tbl_blk = (pd.DataFrame(cov_rows)
                 .pivot_table(index=["Panel", "Block", "Start", "End"],
                              columns="Nominal", values="Empirical")
                 .sort_index())
gap_df      = pd.DataFrame(gap_rows)
gap_tbl_blk = (gap_df.set_index(["Panel", "Block", "Start", "End"])
                        .sort_index())

print(cov_tbl_blk)
print("\nMean‑absolute gap by block:")
print(gap_tbl_blk)

# ---------- ACCURACY × CONSISTENCY score ----------
score_rows = []
for p, grp in gap_df.groupby("Panel"):
    mag_all = grp.loc[grp["Block"] == "All", "AvgAbsGap"].item()
    sd_blk  = grp.loc[grp["Block"].isin([1, 2, 3, 4]), "AvgAbsGap"].std(ddof=1)
    score_rows.append({"Panel": p,
                       "MAG_All": mag_all,
                       "SD_blocks": sd_blk,
                       f"Score(λ={LAMBDA})": mag_all + LAMBDA * sd_blk})

score_tbl = pd.DataFrame(score_rows).set_index("Panel")

print("\nAccuracy × Consistency summary (lower = better):")
print(score_tbl)


# live CI (if applicable)
print("\n=== Live CI using best spec ===")
for p_label, panel in PANELS.items():
    unreleased = panel[panel["actual"].isna()].sort_values("release_date")
    if unreleased.empty:
        print(f"{p_label}: no unreleased month.")
        continue

    row      = unreleased.iloc[-1]
    t_date   = row["release_date"].date()
    pt_med   = row["median_forecast"]
    sp_t     = row["forecast"].std()

    # history window
    hist = (panel[(panel["release_date"] < row["release_date"]) &
                  panel["actual"].notna()]
            .groupby("release_date")
            .agg(median_fc=("median_forecast","first"),
                 actual    =("actual","first"),
                 spread    =("forecast","std"))
            .tail(ROLL_WIN if best_cfg['wtype']=="rolling" else None))

    if len(hist) < MIN_TRAIN:
        print(f"{p_label}: not enough history for live CI.")
        continue

    hist["err"] = hist["median_fc"] - hist["actual"]
    nu, mu, sig = st.t.fit(hist["err"].values)

    centre = pt_med + (mu if best_cfg['use_mu'] else 0.0)
    mult   = crisis_mult(hist["spread"].values, sp_t, best_cfg['crisis'])

    print(f"{p_label} | {t_date}  median = {pt_med:,.0f} k")
    for L in (.50,.60,.70,.80,.90,.95):
        h = ci_half(L, nu, sig)*mult
        print(f"  {int(L*100)}% band : [{centre-h:,.0f} , {centre+h:,.0f}] k")
    print()


COVID | Exp_noShift_No:   0%|          | 0/194 [00:00<?, ?it/s]

COVID | Exp_noShift_Yes:   0%|          | 0/194 [00:00<?, ?it/s]

COVID | Roll36_noShift_No:   0%|          | 0/194 [00:00<?, ?it/s]

COVID | Roll36_noShift_Yes:   0%|          | 0/194 [00:00<?, ?it/s]

COVID | Roll36_plusMu_No:   0%|          | 0/194 [00:00<?, ?it/s]

...

Full | Blk1:   0%|          | 0/73 [00:00<?, ?it/s]

Full | Blk2:   0%|          | 0/73 [00:00<?, ?it/s]

Full | Blk3:   0%|          | 0/73 [00:00<?, ?it/s]

Full | Blk4:   0%|          | 0/72 [00:00<?, ?it/s]

Nominal                                0.50      0.60      0.70      0.80  \
Panel Block Start      End                                                  
COVID 1     2006-06-02 2010-06-04  0.551020  0.591837  0.673469  0.734694   
      2     2010-07-02 2014-07-03  0.530612  0.591837  0.673469  0.795918   
      3     2014-08-01 2018-07-06  0.500000  0.666667  0.708333  0.770833   
      4     2018-08-03 2025-07-03  0.458333  0.520833  0.645833  0.791667   
      All   2003-06-06 2025-07-03  0.510309  0.592784  0.675258  0.773196   
Full  1     2001-05-04 2007-05-04  0.547945  0.643836  0.739726  0.849315   
      2     2007-06-01 2013-06-07  0.493151  0.547945  0.643836  0.726027   
      3     2013-07-05 2019-07-05  0.479452  0.602740  0.671233  0.753425   
      4     2019-08-02 2025-07-03  0.500000  0.569444  0.666667  0.819444   
      All   1997-08-01 2025-07-03  0.505155  0.591065  0.680412  0.786942   

Nominal                                0.90      0.95  
Panel Block Start  

### 2.2 GARCH(1,1)

TODO: consider whether or not to implement expanding (better justified?) vs rolling 

In [47]:
# ───────── global settings ─────────────────────────────────────
warnings.filterwarnings("ignore", category=ConvergenceWarning)
np.seterr(all="ignore")

ROLL_WIN   = 36                       # rolling window length
MIN_TRAIN  = 36                       # first release scored
LEVELS     = np.array([.50,.60,.70,.80,.90,.95])
BETA_BASE  = 0.0                      # crisis spread‑elasticity
BETA_CRIS  = 0.80
PCTL_THRES = 0.95                     # top‑5 % spread ⇒ crisis
LAMBDA     = 1.0                      # weight for consistency score
SCALE      = 1_000.0                  # k‑jobs → hundreds
PANELS     = {"COVID": df, "Full": df_full}

# ───────── helpers ─────────────────────────────────────────────
def ci_half(level: float, nu: float, sigma: float) -> float:
    """Half‑width of two‑sided Student‑t interval (ν = nu, σ = sigma)."""
    return st.t.ppf(1 - (1 - level) / 2, df=nu) * sigma

def garch_one_step(rescaled_errs: np.ndarray):
    """
    Fit Constant‑Mean + GARCH(1,1)‑t on *rescaled* errors and return
    one‑step‑ahead (ν, σ̂ₜ₊₁, μ̂ₜ₊₁) in ORIGINAL units.
    """
    mdl = ConstantMean(rescaled_errs, rescale=False)
    mdl.volatility   = GARCH(1, 1)
    mdl.distribution = StudentsT()
    res = mdl.fit(disp="off")
    fc  = res.forecast(horizon=1)
    mu  = fc.mean.iloc[-1, 0] * SCALE
    sig = math.sqrt(fc.variance.iloc[-1, 0]) * SCALE
    nu  = res.params["nu"]
    return nu, sig, mu

def crisis_mult(sp_hist, sp_t, crisis_flag: bool) -> float:
    """Spread‑based crisis multiplier (1 if crisis_flag is False)."""
    if not crisis_flag:
        return 1.0
    med_sp = np.median(sp_hist)
    pct    = (sp_hist < sp_t).mean()
    beta   = BETA_BASE if pct < PCTL_THRES else BETA_CRIS
    return (sp_t / med_sp) ** beta

# ================================================================
# BACK‑TEST – both panels, two specs (NoAdj / CrisisAdj)
# ================================================================
cov_rows, gap_rows = [], []

for p_label, panel in PANELS.items():
    # ─── per‑release aggregates ────────────────────────────────
    rel = (panel.groupby("release_date")
                 .agg(median_fc=("median_forecast", "first"),
                      actual    =("actual",          "first"),
                      spread    =("forecast",        "std"))
                 .reset_index()
                 .sort_values("release_date"))
    rel = rel[rel["actual"].notna()].reset_index(drop=True)
    rel["err"] = rel["median_fc"] - rel["actual"]

    for crisis_flag in (False, True):
        tag   = f"GARCH_{'CrisisAdj' if crisis_flag else 'NoAdj'}"
        hits  = defaultdict(int); totN = 0

        for i in tqdm(range(MIN_TRAIN, len(rel)),
                      desc=f"{p_label} | {tag}", leave=False):
            # ─── training window ────────────────────────────────
            start = max(0, i - ROLL_WIN) 
            errs  = rel.loc[start:i-1, "err"].values / SCALE
            nu, sig_fc, mu_fc = garch_one_step(errs)

            centre   = rel.at[i, "median_fc"] + mu_fc   # μ‑shift always on
            actual   = rel.at[i, "actual"]
            sp_t     = rel.at[i, "spread"]
            mult     = crisis_mult(rel.loc[start:i-1, "spread"].values,
                                   sp_t, crisis_flag)

            for L in LEVELS:
                if centre - ci_half(L, nu, sig_fc)*mult <= actual <= \
                   centre + ci_half(L, nu, sig_fc)*mult:
                    hits[L] += 1
            totN += 1

        # ─── store results ─────────────────────────────────────
        emp_vec = np.array([hits[L] / totN for L in LEVELS])
        for L, e in zip(LEVELS, emp_vec):
            cov_rows.append(dict(Panel=p_label, Spec=tag,
                                 Nominal=L, Empirical=e))
        gap_rows.append(dict(Panel=p_label, Spec=tag,
                             AvgAbsGap=float(np.abs(emp_vec - LEVELS).mean())))

# ─── tidy & print back‑test tables ─────────────────────────────
cov_tbl = (pd.DataFrame(cov_rows)
             .pivot_table(index=["Panel", "Spec"],
                          columns="Nominal", values="Empirical")
             .sort_index())
gap_tbl = (pd.DataFrame(gap_rows)
             .set_index(["Panel", "Spec"])
             .sort_index())

pd.set_option("display.float_format", "{:.3f}".format)
print("\n=== Back‑test coverage (GARCH(1,1)‑t) ===")
print(cov_tbl)
print("\nMean‑absolute coverage gap (lower = better):")
print(gap_tbl)

# ================================================================
# Pick best spec on FULL panel
# ================================================================
best_row = (gap_tbl.xs("Full")
                     .sort_values("AvgAbsGap")
                     .iloc[0])
best_tag  = best_row.name
best_cfg  = dict(crisis = "CrisisAdj" in best_tag)

print("\n>>> Best spec on FULL panel <<<")
print(f"  Tag        : {best_tag}")
print(f"  CrisisAdj  : {best_cfg['crisis']}")
print(f"  AvgAbsGap  : {best_row['AvgAbsGap']:.4f}")

# ================================================================
# STRATIFIED ANALYSIS  (using best spec)
# ================================================================
print("\n=== Stratified 4‑block results (best spec) ===")
cov_rows, gap_rows = [], []

for p_label, panel in PANELS.items():
    rel = (panel.groupby("release_date")
                 .agg(median_fc=("median_forecast", "first"),
                      actual    =("actual",          "first"),
                      spread    =("forecast",        "std"))
                 .reset_index()
                 .sort_values("release_date"))
    rel = rel[rel["actual"].notna()].reset_index(drop=True)
    rel["err"] = rel["median_fc"] - rel["actual"]

    idx_eval = np.arange(MIN_TRAIN, len(rel))
    blocks   = np.array_split(idx_eval, 4)

    g_hits = defaultdict(int); g_tot = 0

    for b_no, idx in enumerate(blocks, 1):
        hits = defaultdict(int); tot = 0
        st_dt = rel.at[idx[0], "release_date"].date()
        en_dt = rel.at[idx[-1], "release_date"].date()

        for j in tqdm(idx, desc=f"{p_label} | Blk{b_no}", leave=False):
            start = max(0, j - ROLL_WIN)
            errs  = rel.loc[start:j-1, "err"].values / SCALE
            nu, sig_fc, mu_fc = garch_one_step(errs)

            centre = rel.at[j, "median_fc"] + mu_fc
            actual = rel.at[j, "actual"]
            sp_t   = rel.at[j, "spread"]
            mult   = crisis_mult(rel.loc[start:j-1, "spread"].values,
                                 sp_t, best_cfg['crisis'])

            for L in LEVELS:
                hit = centre - ci_half(L, nu, sig_fc)*mult <= actual <= \
                      centre + ci_half(L, nu, sig_fc)*mult
                hits[L]   += hit
                g_hits[L] += hit
            tot += 1
            g_tot += 1

        emp_vec = np.array([hits[L] / tot for L in LEVELS])
        cov_rows.extend([dict(Panel=p_label, Block=b_no,
                              Start=st_dt, End=en_dt,
                              Nominal=L, Empirical=e)
                         for L, e in zip(LEVELS, emp_vec)])
        gap_rows.append(dict(Panel=p_label, Block=b_no,
                             Start=st_dt, End=en_dt,
                             AvgAbsGap=float(np.abs(emp_vec - LEVELS).mean())))

    # global “All”
    emp_all = np.array([g_hits[L] / g_tot for L in LEVELS])
    cov_rows.extend([dict(Panel=p_label, Block="All",
                          Start=rel.iloc[0]["release_date"].date(),
                          End  =rel.iloc[-1]["release_date"].date(),
                          Nominal=L, Empirical=e)
                     for L, e in zip(LEVELS, emp_all)])
    gap_rows.append(dict(Panel=p_label, Block="All",
                         Start=rel.iloc[0]["release_date"].date(),
                         End  =rel.iloc[-1]["release_date"].date(),
                         AvgAbsGap=float(np.abs(emp_all - LEVELS).mean())))

cov_tbl_blk = (pd.DataFrame(cov_rows)
                 .pivot_table(index=["Panel","Block","Start","End"],
                              columns="Nominal", values="Empirical")
                 .sort_index())
gap_df      = pd.DataFrame(gap_rows)
gap_tbl_blk = (gap_df.set_index(["Panel","Block","Start","End"])
                        .sort_index())

print(cov_tbl_blk)
print("\nMean‑absolute gap by block:")
print(gap_tbl_blk)

# ─── Accuracy × Consistency score ──────────────────────────────
score_rows = []
for p, grp in gap_df.groupby("Panel"):
    mag_all = grp.loc[grp["Block"]=="All", "AvgAbsGap"].item()
    sd_blk  = grp.loc[grp["Block"].isin([1,2,3,4]), "AvgAbsGap"].std(ddof=1)
    score_rows.append({"Panel": p,
                       "MAG_All": mag_all,
                       "SD_blocks": sd_blk,
                       f"Score(λ={LAMBDA})": mag_all + LAMBDA*sd_blk})
score_tbl = pd.DataFrame(score_rows).set_index("Panel")

print("\nAccuracy × Consistency summary (lower = better):")
print(score_tbl)

# ================================================================
# LIVE CI for unreleased month (best spec)
# ================================================================
print("\n=== Live CI using best GARCH spec ===")
for p_label, panel in PANELS.items():
    unreleased = panel[panel["actual"].isna()].sort_values("release_date")
    if unreleased.empty:
        print(f"{p_label}: no unreleased month.")
        continue

    row    = unreleased.iloc[-1]
    t_date = row["release_date"].date()
    pt_med = row["median_forecast"]
    sp_t   = row["forecast"].std()

    # history window
    hist = (panel[(panel["release_date"] < row["release_date"]) &
                  panel["actual"].notna()]
            .groupby("release_date")
            .agg(median_fc=("median_forecast","first"),
                 actual    =("actual","first"),
                 spread    =("forecast","std"))
            .tail(ROLL_WIN))
    if len(hist) < MIN_TRAIN:
        print(f"{p_label}: not enough history for live CI.")
        continue

    hist["err"] = hist["median_fc"] - hist["actual"]
    nu, sig_fc, mu_fc = garch_one_step(hist["err"].values / SCALE)

    centre = pt_med + mu_fc
    mult   = crisis_mult(hist["spread"].values, sp_t, best_cfg['crisis'])

    print(f"{p_label} | {t_date}  median = {pt_med:,.0f} k")
    for L in LEVELS:
        h = ci_half(L, nu, sig_fc) * mult
        print(f"  {int(L*100)}% band : [{centre-h:,.0f} , {centre+h:,.0f}] k")
    print()


COVID | GARCH_NoAdj:   0%|          | 0/194 [00:00<?, ?it/s]

c:\Users\nwong\AppData\Local\Programs\Python\Python313\Lib\site-packages\arch\univariate\base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


COVID | GARCH_CrisisAdj:   0%|          | 0/194 [00:00<?, ?it/s]

c:\Users\nwong\AppData\Local\Programs\Python\Python313\Lib\site-packages\arch\univariate\base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


Full | GARCH_NoAdj:   0%|          | 0/291 [00:00<?, ?it/s]

...

c:\Users\nwong\AppData\Local\Programs\Python\Python313\Lib\site-packages\arch\univariate\base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


Full | Blk2:   0%|          | 0/73 [00:00<?, ?it/s]

Full | Blk3:   0%|          | 0/73 [00:00<?, ?it/s]

Full | Blk4:   0%|          | 0/72 [00:00<?, ?it/s]

Nominal                            0.500  0.600  0.700  0.800  0.900  0.950
Panel Block Start      End                                                 
COVID 1     2006-06-02 2010-06-04  0.469  0.531  0.633  0.776  0.857  0.898
      2     2010-07-02 2014-07-03  0.490  0.633  0.653  0.776  0.918  0.959
      3     2014-08-01 2018-07-06  0.521  0.646  0.708  0.792  0.875  0.958
      4     2018-08-03 2025-07-03  0.521  0.583  0.708  0.792  0.896  0.938
      All   2003-06-06 2025-07-03  0.500  0.598  0.675  0.784  0.887  0.938
Full  1     2001-05-04 2007-05-04  0.425  0.534  0.644  0.726  0.808  0.877
      2     2007-06-01 2013-06-07  0.493  0.603  0.658  0.795  0.904  0.945
      3     2013-07-05 2019-07-05  0.466  0.589  0.658  0.753  0.849  0.918
      4     2019-08-02 2025-07-03  0.611  0.708  0.792  0.875  0.972  0.972
      All   1997-08-01 2025-07-03  0.498  0.608  0.687  0.787  0.883  0.928

Mean‑absolute gap by block:
                                   AvgAbsGap
Panel Block St

### 2.3 Gaussian Mixture Model

- Prep. series of median residuals 
- Fit a GMM to the chosen window (either trailing 36 releases or full history if expanding window)
- For k = 1...4, we fit a GaussianMixture with full covariance. k = 1: single normal. k > 1: skew + fat tails captured through normal mixture. 
- Select optimal model (hyperparam *k*) with BIC 
- Monte Carlo sample 100k errors. Extract error quantiles to give lower/upper cutoffs
- Center band on median forecast for confidence interval (with/without crisis adjustment) 

In [49]:
# ───────── global settings ─────────────────────────────────────
warnings.filterwarnings("ignore", category=RuntimeWarning)
np.random.seed(7)

ROLL_WIN    = 36                       # rolling‑window length
MIN_TRAIN   = 36                       # first release scored
LEVELS      = np.array([.50,.60,.70,.80,.90,.95])
BETA_BASE   = 0.0                      # crisis spread‑elasticity
BETA_CRIS   = 0.80
PCTL_THRES  = 0.95                     # top‑5 % spread ⇒ crisis
LAMBDA      = 1.0                      # weight for consistency score
N_SIMS      = 100_000                  # MC draws for quantiles
K_GRID      = range(1, 5)              # mixture sizes to try
PANELS      = {"COVID": df, "Full": df_full}

# ───────── helpers ─────────────────────────────────────────────
def best_gmm(errs: np.ndarray) -> GaussianMixture:
    """Fit k=1…4 GMMs; return model with the lowest BIC."""
    errs = errs.reshape(-1, 1)
    best, best_bic = None, math.inf
    for k in K_GRID:
        gm  = GaussianMixture(n_components=k, covariance_type="full",
                              reg_covar=1e-6, random_state=0).fit(errs)
        bic = gm.bic(errs)
        if bic < best_bic:
            best, best_bic = gm, bic
    return best

def mc_quantiles(gm: GaussianMixture, probs, n=N_SIMS):
    """Two‑sided quantiles of a fitted GMM via Monte‑Carlo sampling."""
    sims = gm.sample(n)[0].ravel()
    q_lo = np.quantile(sims, (1 - probs) / 2)           # negative
    q_hi = np.quantile(sims, 1 - (1 - probs) / 2)       # positive
    return q_lo, q_hi                                   # vectors

def crisis_mult(sp_hist, sp_t, crisis_flag: bool) -> float:
    """Spread‑based crisis multiplier (1 if crisis_flag is False)."""
    if not crisis_flag:
        return 1.0
    med_sp = np.median(sp_hist)
    pct    = (sp_hist < sp_t).mean()
    beta   = BETA_BASE if pct < PCTL_THRES else BETA_CRIS
    return (sp_t / med_sp) ** beta

# =================================================================
# BACKTEST  –  Expanding vs Roll‑36  ×  (NoAdj / CrisisAdj)
# =================================================================
SPEC_GRID = [("Expanding", False), ("Expanding", True),
             ("Roll36",   False), ("Roll36",   True)]

cov_rows, gap_rows = [], []

for p_label, panel in PANELS.items():
    # ─── per‑release aggregates ────────────────────────────────
    rel = (panel.groupby("release_date")
                 .agg(median_fc=("median_forecast", "first"),
                      actual    =("actual",          "first"),
                      spread    =("forecast",        "std"))
                 .reset_index()
                 .sort_values("release_date"))
    rel = rel[rel["actual"].notna()].reset_index(drop=True)
    rel["err"] = rel["median_fc"] - rel["actual"]

    for wtype, crisis in SPEC_GRID:
        tag   = f"{wtype}_{'CrisisAdj' if crisis else 'NoAdj'}"
        hits  = defaultdict(int); totN = 0

        for i in tqdm(range(len(rel)), desc=f"{p_label} | {tag}", leave=False):
            # warm‑up check
            if (wtype == "Expanding" and i < MIN_TRAIN) or \
               (wtype == "Roll36"   and i < ROLL_WIN):
                continue

            # ─── training errors ───────────────────────────────
            if wtype == "Expanding":
                err_hist = rel.loc[:i-1, "err"].values
                sp_hist  = rel.loc[:i-1, "spread"].values
            else:  # Roll‑36
                err_hist = rel.loc[i-ROLL_WIN:i-1, "err"].values
                sp_hist  = rel.loc[i-ROLL_WIN:i-1, "spread"].values
            if err_hist.size < MIN_TRAIN:
                continue

            gmm        = best_gmm(err_hist)
            q_lo, q_hi = mc_quantiles(gmm, LEVELS)

            centre   = rel.at[i, "median_fc"]           # no μ‑shift in GMM (MC => implicitly centered at mu)
            actual   = rel.at[i, "actual"]
            spread_t = rel.at[i, "spread"]
            mult     = crisis_mult(sp_hist, spread_t, crisis)

            lo_band  = centre - mult * q_hi
            hi_band  = centre - mult * q_lo

            for L, lo, hi in zip(LEVELS, lo_band, hi_band):
                if lo <= actual <= hi:
                    hits[L] += 1
            totN += 1

        if totN == 0:  continue
        emp_vec = np.array([hits[L]/totN for L in LEVELS])
        for L, e in zip(LEVELS, emp_vec):
            cov_rows.append(dict(Panel=p_label, Spec=tag,
                                 Nominal=L, Empirical=e))
        gap_rows.append(dict(Panel=p_label, Spec=tag,
                             AvgAbsGap=float(np.abs(emp_vec - LEVELS).mean())))

# ─── tidy & print back‑test tables ─────────────────────────────
cov_tbl = (pd.DataFrame(cov_rows)
             .pivot_table(index=["Panel", "Spec"],
                          columns="Nominal", values="Empirical")
             .sort_index())
gap_tbl = (pd.DataFrame(gap_rows)
             .set_index(["Panel", "Spec"])
             .sort_index())

pd.set_option("display.float_format", "{:.3f}".format)
print("\n=== Back‑test coverage (Gaussian‑Mixture) ===")
print(cov_tbl)
print("\nMean‑absolute coverage gap (lower = better):")
print(gap_tbl)

# =================================================================
# Pick best spec on FULL panel
# =================================================================
best_row = (gap_tbl.xs("Full")
                     .sort_values("AvgAbsGap")
                     .iloc[0])
best_tag = best_row.name
best_cfg = {
    "wtype"  : best_tag.split("_")[0],           # Expanding / Roll36
    "crisis" : "CrisisAdj" in best_tag
}

print("\n>>> Best spec on FULL panel <<<")
print(f"  Tag        : {best_tag}")
print(f"  Window     : {best_cfg['wtype']}")
print(f"  CrisisAdj  : {best_cfg['crisis']}")
print(f"  AvgAbsGap  : {best_row['AvgAbsGap']:.4f}")

# =================================================================
# STRATIFIED ANALYSIS  (on best spec)
# =================================================================
print("\n=== Stratified 4‑block results (best spec) ===")
cov_rows, gap_rows = [], []

for p_label, panel in PANELS.items():
    rel = (panel.groupby("release_date")
                 .agg(median_fc=("median_forecast", "first"),
                      actual    =("actual",          "first"),
                      spread    =("forecast",        "std"))
                 .reset_index()
                 .sort_values("release_date"))
    rel = rel[rel["actual"].notna()].reset_index(drop=True)
    rel["err"] = rel["median_fc"] - rel["actual"]

    idx_eval = np.arange(MIN_TRAIN, len(rel))
    blocks   = np.array_split(idx_eval, 4)

    g_hits = defaultdict(int); g_tot = 0

    for b_no, idx in enumerate(blocks, 1):
        hits = defaultdict(int); tot = 0
        st_dt = rel.at[idx[0], "release_date"].date()
        en_dt = rel.at[idx[-1], "release_date"].date()

        for j in tqdm(idx, desc=f"{p_label} | Blk{b_no}", leave=False):
            # training errors per spec ---------------------------
            if best_cfg['wtype'] == "Expanding":
                err_hist = rel.loc[:j-1, "err"].values
                sp_hist  = rel.loc[:j-1, "spread"].values
            else:
                if j < ROLL_WIN:  continue
                err_hist = rel.loc[j-ROLL_WIN:j-1, "err"].values
                sp_hist  = rel.loc[j-ROLL_WIN:j-1, "spread"].values
            if err_hist.size < MIN_TRAIN:
                continue

            gmm        = best_gmm(err_hist)
            q_lo, q_hi = mc_quantiles(gmm, LEVELS)

            centre = rel.at[j, "median_fc"]
            actual = rel.at[j, "actual"]
            sp_t   = rel.at[j, "spread"]
            mult   = crisis_mult(sp_hist, sp_t, best_cfg['crisis'])

            lo_band = centre - mult * q_hi
            hi_band = centre - mult * q_lo

            for L, lo, hi in zip(LEVELS, lo_band, hi_band):
                hit = lo <= actual <= hi
                hits[L]   += hit
                g_hits[L] += hit
            tot += 1
            g_tot += 1

        emp_vec = np.array([hits[L] / tot for L in LEVELS])
        cov_rows.extend([dict(Panel=p_label, Block=b_no,
                              Start=st_dt, End=en_dt,
                              Nominal=L, Empirical=e)
                         for L, e in zip(LEVELS, emp_vec)])
        gap_rows.append(dict(Panel=p_label, Block=b_no,
                             Start=st_dt, End=en_dt,
                             AvgAbsGap=float(np.abs(emp_vec - LEVELS).mean())))

    # global “All” row
    emp_all = np.array([g_hits[L] / g_tot for L in LEVELS])
    cov_rows.extend([dict(Panel=p_label, Block="All",
                          Start=rel.iloc[0]["release_date"].date(),
                          End  =rel.iloc[-1]["release_date"].date(),
                          Nominal=L, Empirical=e)
                     for L, e in zip(LEVELS, emp_all)])
    gap_rows.append(dict(Panel=p_label, Block="All",
                         Start=rel.iloc[0]["release_date"].date(),
                         End  =rel.iloc[-1]["release_date"].date(),
                         AvgAbsGap=float(np.abs(emp_all - LEVELS).mean())))

cov_tbl_blk = (pd.DataFrame(cov_rows)
                 .pivot_table(index=["Panel","Block","Start","End"],
                              columns="Nominal", values="Empirical")
                 .sort_index())
gap_df      = pd.DataFrame(gap_rows)
gap_tbl_blk = (gap_df.set_index(["Panel","Block","Start","End"])
                        .sort_index())

print(cov_tbl_blk)
print("\nMean‑absolute gap by block:")
print(gap_tbl_blk)

# ─── Accuracy × Consistency score ──────────────────────────────
score_rows = []
for p, grp in gap_df.groupby("Panel"):
    mag_all = grp.loc[grp["Block"]=="All", "AvgAbsGap"].item()
    sd_blk  = grp.loc[grp["Block"].isin([1,2,3,4]), "AvgAbsGap"].std(ddof=1)
    score_rows.append({"Panel": p,
                       "MAG_All": mag_all,
                       "SD_blocks": sd_blk,
                       f"Score(λ={LAMBDA})": mag_all + LAMBDA*sd_blk})
score_tbl = pd.DataFrame(score_rows).set_index("Panel")

print("\nAccuracy × Consistency summary (lower = better):")
print(score_tbl)

# =================================================================
# 4) LIVE CI for unreleased month (best spec)
# =================================================================
print("\n=== Live CI using best GMM spec ===")
for p_label, panel in PANELS.items():
    unreleased = panel[panel["actual"].isna()].sort_values("release_date")
    if unreleased.empty:
        print(f"{p_label}: no unreleased month.")
        continue

    row    = unreleased.iloc[-1]
    t_date = row["release_date"].date()
    pt_med = row["median_forecast"]
    sp_t   = row["forecast"].std()

    # history window ---------------------------------------------
    hist = (panel[(panel["release_date"] < row["release_date"]) &
                  panel["actual"].notna()]
            .groupby("release_date")
            .agg(median_fc=("median_forecast","first"),
                 actual    =("actual","first"),
                 spread    =("forecast","std")))
    if best_cfg['wtype'] == "Roll36":
        hist = hist.tail(ROLL_WIN)
    if len(hist) < MIN_TRAIN:
        print(f"{p_label}: not enough history for live CI.")
        continue

    err_hist = (hist["median_fc"] - hist["actual"]).values
    gmm      = best_gmm(err_hist)
    q_lo, q_hi = mc_quantiles(gmm, LEVELS)

    mult   = crisis_mult(hist["spread"].values, sp_t, best_cfg['crisis'])
    lo_b   = pt_med - mult * q_hi
    hi_b   = pt_med - mult * q_lo

    print(f"{p_label} | {t_date}  median = {pt_med:,.0f} k")
    for L, lo, hi in zip(LEVELS, lo_b, hi_b):
        print(f"  {int(L*100)}% band : [{lo:,.0f} , {hi:,.0f}] k")
    print()


COVID | Expanding_NoAdj:   0%|          | 0/230 [00:00<?, ?it/s]

COVID | Expanding_CrisisAdj:   0%|          | 0/230 [00:00<?, ?it/s]

COVID | Roll36_NoAdj:   0%|          | 0/230 [00:00<?, ?it/s]

COVID | Roll36_CrisisAdj:   0%|          | 0/230 [00:00<?, ?it/s]

Full | Expanding_NoAdj:   0%|          | 0/327 [00:00<?, ?it/s]

...

Full | Blk1:   0%|          | 0/73 [00:00<?, ?it/s]

Full | Blk2:   0%|          | 0/73 [00:00<?, ?it/s]

Full | Blk3:   0%|          | 0/73 [00:00<?, ?it/s]

Full | Blk4:   0%|          | 0/72 [00:00<?, ?it/s]

Nominal                            0.500  0.600  0.700  0.800  0.900  0.950
Panel Block Start      End                                                 
COVID 1     2006-06-02 2010-06-04  0.531  0.653  0.673  0.755  0.918  0.959
      2     2010-07-02 2014-07-03  0.531  0.612  0.694  0.776  0.980  0.980
      3     2014-08-01 2018-07-06  0.500  0.625  0.708  0.771  0.833  0.917
      4     2018-08-03 2025-07-03  0.458  0.604  0.708  0.771  0.854  0.917
      All   2003-06-06 2025-07-03  0.505  0.624  0.696  0.768  0.897  0.943
Full  1     2001-05-04 2007-05-04  0.616  0.740  0.795  0.849  0.918  0.959
      2     2007-06-01 2013-06-07  0.493  0.603  0.658  0.740  0.945  0.973
      3     2013-07-05 2019-07-05  0.479  0.575  0.671  0.740  0.836  0.890
      4     2019-08-02 2025-07-03  0.542  0.597  0.708  0.792  0.889  0.944
      All   1997-08-01 2025-07-03  0.533  0.629  0.708  0.780  0.897  0.942

Mean‑absolute gap by block:
                                   AvgAbsGap
Panel Block St

### 2.4 Bayesian Model Averaging
1. Compute error series for median forecast 
2. Per window, we fit two candidate errors models. 
    - Normal, fit via closed form MLE. Compute BIC
    - Student-t. Numerical MLE fit for params. Get BIC
3. Convert BICs from 2 models fit to Bayesian model weights via Occam weights. A lower BIC will yield a higher posterior probability. 
4. Similar to GMM: Monte Carlo 100,000 trials. Pick each error from either a normal and student-t distribution with probabilities determined by weights above. 
5. From simulated error bands, convert to forecast bands centered around median

In [57]:
# ---------- global settings -----------------------------------
warnings.filterwarnings("ignore", category=RuntimeWarning)
np.random.seed(7)                                 # reproducibility

ROLL_WIN   = 36                       # rolling window length
MIN_TRAIN  = 36                       # first release scored
LEVELS     = np.array([.50, .60, .70, .80, .90, .95])
BETA_BASE  = 0.0                      # crisis spread-elasticity
BETA_CRIS  = 0.80
PCTL_THRES = 0.95                     # top-5 % spread => crisis
LAMBDA     = 1.0                      # weight for consistency score
PANELS     = {"COVID": df, "Full": df_full}

N_SIMS     = 100_000                  # Monte-Carlo draws per fit
RNG        = np.random.default_rng(42)  # single RNG for whole cell

# ---------- helpers -------------------------------------------
def fit_normal(errs):
    mu  = np.mean(errs)
    sigma  = max(np.std(errs, ddof=1), 1e-6)      # guard sigma->0
    ll = np.sum(norm.logpdf(errs, loc=mu, scale=sigma))
    bic = 2 * np.log(len(errs)) - 2 * ll          # k = 2
    return {"name": "Normal", "mu": mu, "sigma": sigma, "nu": None,
            "ll": ll, "bic": bic}

def fit_student_t(errs):
    mu0, sigma0 = np.mean(errs), max(np.std(errs, ddof=1), 1e-6)
    def nll(p):
        nu, mu, sigma = p
        if nu <= 2 or sigma <= 1e-8:
            return np.inf
        return -np.sum(student_t.logpdf(errs, df=nu, loc=mu, scale=sigma))
    res = minimize(nll, (5.0, mu0, sigma0),
                   bounds=[(2.1, 100), (None, None), (1e-6, None)])
    nu, mu, sigma = res.x
    ll  = -res.fun
    bic = 3 * np.log(len(errs)) - 2 * ll          # k = 3
    return {"name": "Student-t", "mu": mu, "sigma": sigma, "nu": nu,
            "ll": ll, "bic": bic}

def bma_models(errs):
    """
    Fit Normal & Student-t; return list with weights ~ exp(-1/2 * deltaBIC).
    Guarantees the weights are finite and sum to 1.
    """
    m_n, m_t  = fit_normal(errs), fit_student_t(errs)
    bic_min   = min(m_n["bic"], m_t["bic"])
    w_n       = math.exp(-0.5 * (m_n["bic"] - bic_min))
    w_t       = math.exp(-0.5 * (m_t["bic"] - bic_min))
    z         = w_n + w_t
    if not np.isfinite(z) or z == 0:              # degenerate -> equal weights
        w_n, w_t, z = 0.5, 0.5, 1.0
    m_n["w"], m_t["w"] = w_n / z, w_t / z
    return [m_n, m_t]

def draw_bma(models, n_draws, *, rng):
    """Monte-Carlo draws from BMA mixture."""
    p = np.array([m["w"] for m in models])
    if np.isnan(p).any() or p.sum() == 0:         # final safety net
        p = np.full_like(p, 1 / len(p))
    picks = rng.choice(len(models), size=n_draws, p=p)
    draws = np.empty(n_draws)
    for idx, m in enumerate(models):
        m_n = np.sum(picks == idx)
        if m_n == 0:
            continue
        if m["name"] == "Normal":
            draws[picks == idx] = rng.normal(m["mu"], m["sigma"], size=m_n)
        else:                                     # Student-t
            draws[picks == idx] = (
                m["mu"] + m["sigma"] * rng.standard_t(m["nu"], size=m_n)
            )
    return draws

def mc_quantiles_bma(models, probs):
    sims = draw_bma(models, N_SIMS, rng=RNG)
    q_lo = np.quantile(sims, (1 - probs) / 2)
    q_hi = np.quantile(sims, 1 - (1 - probs) / 2)
    return q_lo, q_hi                             # vectors

def crisis_mult(sp_hist, sp_t, crisis_flag):
    """Return 1 if crisis_flag==False; else spread-based multiplier."""
    if not crisis_flag:
        return 1.0
    med_sp = np.median(sp_hist)
    pct    = (sp_hist < sp_t).mean()
    beta   = BETA_BASE if pct < PCTL_THRES else BETA_CRIS
    return (sp_t / med_sp) ** beta

# ---------- spec grid  (window x crisis flag) -----------------
SPEC_GRID = [("Expanding", False), ("Expanding", True),
             ("Roll36",   False), ("Roll36",   True)]
spec_to_cfg = {f"{w}_{'CrisisAdj' if c else 'NoAdj'}":
               dict(window=w, crisis=c) for w, c in SPEC_GRID}

# ==============================================================
# BACK-TEST  (both panels, all tags)
# ==============================================================
cov_rows, gap_rows = [], []

for p_label, panel in PANELS.items():
    rel = (panel.groupby("release_date")
                 .agg(median_fc=("median_forecast", "first"),
                      actual    =("actual", "first"),
                      spread    =("forecast", "std"))
                 .reset_index()
                 .sort_values("release_date"))
    rel = rel.dropna(subset=["actual"]).reset_index(drop=True)
    rel["err"] = rel["median_fc"] - rel["actual"]

    for window, crisis in SPEC_GRID:
        tag   = f"{window}_{'CrisisAdj' if crisis else 'NoAdj'}"
        hits  = defaultdict(int)
        totN  = 0

        for i in tqdm(range(len(rel)), desc=f"{p_label} | {tag}", leave=False):
            if (window == "Expanding" and i < MIN_TRAIN) or \
               (window == "Roll36"    and i < ROLL_WIN):
                continue
            start = 0 if window == "Expanding" else i - ROLL_WIN
            hist_errs = rel.loc[start:i-1, "err"].values
            if hist_errs.size < MIN_TRAIN or hist_errs.std() < 1e-8:
                continue                              # skip degenerate window
            models = bma_models(hist_errs)
            q_lo, q_hi = mc_quantiles_bma(models, LEVELS)

            centre = rel.at[i, "median_fc"]           # no mu-shift
            mult   = crisis_mult(rel.loc[start:i-1, "spread"].values,
                                 rel.at[i, "spread"], crisis)
            band_lo = centre - mult * q_hi
            band_hi = centre - mult * q_lo
            actual  = rel.at[i, "actual"]

            for L, lo, hi in zip(LEVELS, band_lo, band_hi):
                if lo <= actual <= hi:
                    hits[L] += 1
            totN += 1

        if not totN:
            continue
        emp_vec = np.array([hits[L] / totN for L in LEVELS])
        for L, e in zip(LEVELS, emp_vec):
            cov_rows.append(dict(Panel=p_label, Spec=tag,
                                 Nominal=L, Empirical=e))
        gap_rows.append(dict(Panel=p_label, Spec=tag,
                             AvgAbsGap=float(np.abs(emp_vec - LEVELS).mean())))

cov_tbl = (pd.DataFrame(cov_rows)
             .pivot_table(index=["Panel", "Spec"],
                          columns="Nominal", values="Empirical")
             .sort_index())
gap_tbl = (pd.DataFrame(gap_rows)
             .set_index(["Panel", "Spec"])
             .sort_index())

pd.set_option("display.float_format", "{:.3f}".format)
print("\n=== Back-test coverage (BMA) ===")
print(cov_tbl)
print("\nMean-absolute coverage gap (lower = better):")
print(gap_tbl)

# ==============================================================
# Pick best spec
# ==============================================================
best_row = (gap_tbl.xs("Full")
                     .sort_values("AvgAbsGap")
                     .iloc[0])
best_tag = best_row.name
best_cfg = spec_to_cfg[best_tag]

print("\n>>> Best spec on FULL panel <<<")
print(f"  Tag        : {best_tag}")
print(f"  Window     : {best_cfg['window']}")
print(f"  CrisisAdj  : {best_cfg['crisis']}")
print(f"  AvgAbsGap  : {best_row['AvgAbsGap']:.4f}")

# ==============================================================
# STRATIFIED ANALYSIS 
# ==============================================================
print("\n=== Stratified 4-block results (best spec) ===")
cov_rows, gap_rows = [], []

for p_label, panel in PANELS.items():
    rel = (panel.groupby("release_date")
                 .agg(median_fc=("median_forecast", "first"),
                      actual    =("actual", "first"),
                      spread    =("forecast", "std"))
                 .reset_index()
                 .sort_values("release_date"))
    rel = rel.dropna(subset=["actual"]).reset_index(drop=True)
    rel["err"] = rel["median_fc"] - rel["actual"]

    eval_idx = np.arange(MIN_TRAIN, len(rel))
    blocks   = np.array_split(eval_idx, 4)

    g_hits = defaultdict(int)
    g_tot = 0

    for b_no, idx in enumerate(blocks, 1):
        hits = defaultdict(int)
        tot = 0
        st_dt = rel.at[idx[0], "release_date"].date()
        en_dt = rel.at[idx[-1], "release_date"].date()

        for j in tqdm(idx, desc=f"{p_label} | Blk{b_no}", leave=False):
            start = 0 if best_cfg['window'] == "Expanding" else j - ROLL_WIN
            if start < 0 or (j - start) < MIN_TRAIN:
                continue
            hist_errs = rel.loc[start:j-1, "err"].values
            if hist_errs.std() < 1e-8:
                continue
            models = bma_models(hist_errs)
            q_lo, q_hi = mc_quantiles_bma(models, LEVELS)

            centre = rel.at[j, "median_fc"]
            mult   = crisis_mult(rel.loc[start:j-1, "spread"].values,
                                 rel.at[j, "spread"], best_cfg['crisis'])
            band_lo = centre - mult * q_hi
            band_hi = centre - mult * q_lo
            actual  = rel.at[j, "actual"]

            for L, lo, hi in zip(LEVELS, band_lo, band_hi):
                hit = lo <= actual <= hi
                hits[L]   += hit
                g_hits[L] += hit
            tot   += 1
            g_tot += 1

        if tot:
            emp_vec = np.array([hits[L] / tot for L in LEVELS])
            cov_rows.extend([dict(Panel=p_label, Block=b_no,
                                  Start=st_dt, End=en_dt,
                                  Nominal=L, Empirical=e)
                             for L, e in zip(LEVELS, emp_vec)])
            gap_rows.append(dict(Panel=p_label, Block=b_no,
                                 Start=st_dt, End=en_dt,
                                 AvgAbsGap=float(np.abs(emp_vec - LEVELS).mean())))

    if g_tot:
        emp_all = np.array([g_hits[L] / g_tot for L in LEVELS])
        cov_rows.extend([dict(Panel=p_label, Block="All",
                              Start=rel.iloc[0]["release_date"].date(),
                              End  =rel.iloc[-1]["release_date"].date(),
                              Nominal=L, Empirical=e)
                         for L, e in zip(LEVELS, emp_all)])
        gap_rows.append(dict(Panel=p_label, Block="All",
                             Start=rel.iloc[0]["release_date"].date(),
                             End  =rel.iloc[-1]["release_date"].date(),
                             AvgAbsGap=float(np.abs(emp_all - LEVELS).mean())))

cov_tbl_blk = (pd.DataFrame(cov_rows)
                 .pivot_table(index=["Panel", "Block", "Start", "End"],
                              columns="Nominal", values="Empirical")
                 .sort_index())
gap_df      = pd.DataFrame(gap_rows)
gap_tbl_blk = (gap_df.set_index(["Panel", "Block", "Start", "End"])
                        .sort_index())

print(cov_tbl_blk)
print("\nMean-absolute gap by block:")
print(gap_tbl_blk)

# --- Accuracy x Consistency score -----------------------------
score_rows = []
for p, grp in gap_df.groupby("Panel"):
    mag_all = grp.loc[grp["Block"] == "All", "AvgAbsGap"].item()
    sd_blk  = grp.loc[grp["Block"].isin([1, 2, 3, 4]), "AvgAbsGap"].std(ddof=1)
    score_rows.append({"Panel": p,
                       "MAG_All": mag_all,
                       "SD_blocks": sd_blk,
                       f"Score(lambda={LAMBDA})": mag_all + LAMBDA * sd_blk})
score_tbl = pd.DataFrame(score_rows).set_index("Panel")

print("\nAccuracy x Consistency summary (lower = better):")
print(score_tbl)

# ==============================================================
# LIVE CI for unreleased month (best spec)
# ==============================================================
print("\n=== Live CI using best BMA spec ===")
for p_label, panel in PANELS.items():
    unreleased = panel[panel["actual"].isna()].sort_values("release_date")
    if unreleased.empty:
        print(f"{p_label}: no unreleased month.")
        continue

    row    = unreleased.iloc[-1]
    t_date = row["release_date"].date()
    pt_med = row["median_forecast"]
    sp_t   = row["forecast"].std()

    # history window  – aggregate by release to avoid sigma approx 0
    hist = (panel[(panel["release_date"] < row["release_date"]) &
                  panel["actual"].notna()]
            .groupby("release_date")
            .agg(median_fc=("median_forecast", "first"),
                 actual    =("actual", "first"),
                 spread    =("forecast", "std"))
            .tail(ROLL_WIN if best_cfg['window'] == "Roll36" else None))
    if len(hist) < MIN_TRAIN or \
       (hist["median_fc"] - hist["actual"]).std() < 1e-8:
        print(f"{p_label}: not enough usable history for live CI.")
        continue

    hist_errs = (hist["median_fc"] - hist["actual"]).values
    models    = bma_models(hist_errs)
    q_lo, q_hi = mc_quantiles_bma(models, LEVELS)

    mult    = crisis_mult(hist["spread"].values, sp_t, best_cfg['crisis'])
    band_lo = pt_med - mult * q_hi
    band_hi = pt_med - mult * q_lo

    print(f"{p_label} | {t_date}  median = {pt_med:,.0f} k")
    for L, lo, hi in zip(LEVELS, band_lo, band_hi):
        print(f"  {int(L * 100)}% band : [{lo:,.0f} , {hi:,.0f}] k")
    print()


COVID | Expanding_NoAdj:   0%|          | 0/230 [00:00<?, ?it/s]

COVID | Expanding_CrisisAdj:   0%|          | 0/230 [00:00<?, ?it/s]

COVID | Roll36_NoAdj:   0%|          | 0/230 [00:00<?, ?it/s]

COVID | Roll36_CrisisAdj:   0%|          | 0/230 [00:00<?, ?it/s]

Full | Expanding_NoAdj:   0%|          | 0/327 [00:00<?, ?it/s]

...

Full | Blk1:   0%|          | 0/73 [00:00<?, ?it/s]

Full | Blk2:   0%|          | 0/73 [00:00<?, ?it/s]

Full | Blk3:   0%|          | 0/73 [00:00<?, ?it/s]

Full | Blk4:   0%|          | 0/72 [00:00<?, ?it/s]

Nominal                            0.500  0.600  0.700  0.800  0.900  0.950
Panel Block Start      End                                                 
COVID 1     2006-06-02 2010-06-04  0.531  0.673  0.673  0.796  0.918  0.959
      2     2010-07-02 2014-07-03  0.510  0.571  0.694  0.816  0.980  0.980
      3     2014-08-01 2018-07-06  0.500  0.646  0.708  0.771  0.875  0.938
      4     2018-08-03 2025-07-03  0.438  0.646  0.708  0.792  0.875  0.917
      All   2003-06-06 2025-07-03  0.495  0.634  0.696  0.794  0.912  0.948
Full  1     2001-05-04 2007-05-04  0.603  0.753  0.781  0.849  0.918  0.959
      2     2007-06-01 2013-06-07  0.479  0.575  0.644  0.795  0.959  0.973
      3     2013-07-05 2019-07-05  0.479  0.575  0.685  0.740  0.863  0.904
      4     2019-08-02 2025-07-03  0.569  0.625  0.694  0.764  0.889  0.917
      All   1997-08-01 2025-07-03  0.533  0.632  0.701  0.787  0.907  0.938

Mean-absolute gap by block:
                                   AvgAbsGap
Panel Block St

## Appendix: Confidence intervals built around uncertainty in the cross-section

In this section, we test empirically whether accurate confidence intervals can be built using solely cross-sectional spread (standard deviation of forecasts). We also test different rolling window lengths for student-t confidence intervals.

Conclusion: Confidence intervals **cannot** be built solely from cross-sectional forecast spread

In [ ]:
# Buidling confidence intervals using uncertainty in the cross-section (in isolation), try rolling FIXED windows

# -------------------------------------------------------------
# XS-t (centre = μ̂)   &   TS-t  empirical coverage – both panels
# ------------------------------------------------------------
LEVELS  = [0.50, 0.60, 0.70, 0.80, 0.90, 0.95]
TS_WINS = [12, 24, 36, 60, 120]          # months
MIN_XS  = 5                          # min forecasts in cross-section
PANELS  = {"COVID": df, "Full": df_full}

def in_band(center, sig, nu, level, actual):
    half = st.t.ppf(1 - (1-level)/2, df=nu) * sig
    return int(center - half <= actual <= center + half)

rows = []

for panel_name, panel in PANELS.items():

    # cross-section 
    xs_hits = {L: 0 for L in LEVELS};  xs_tot = 0

    for date, grp in tqdm(panel.groupby("release_date"), desc=f"{panel_name} XS"):
        sample = grp["forecast"].dropna().values
        act    = grp["actual"].iloc[0]
        if len(sample) < MIN_XS or np.isnan(act):
            continue
        nu, loc, sig = st.t.fit(sample)           # μ̂ = loc
        for L in LEVELS:
            xs_hits[L] += in_band(loc, sig, nu, L, act)
        xs_tot += 1

    for L in LEVELS:
        rows.append({"Method": f"{panel_name}-XS-t",
                     "Nominal": L,
                     "Empirical": xs_hits[L]/xs_tot if xs_tot else np.nan})

    # time-series
    ts_hits = {w:{L:0 for L in LEVELS} for w in TS_WINS}
    ts_tot  = {w:0 for w in TS_WINS}

    panel_sorted = panel.sort_values("release_date")
    err_series = (panel_sorted.groupby("release_date")
                                .apply(lambda g: g["median_forecast"].iloc[0] - g["actual"].iloc[0])
                                .dropna())
    dates = err_series.index.to_list()

    for win in TS_WINS:
        for i in tqdm(range(win, len(dates)), desc=f"{panel_name} TS {win}m", leave=False):
            train_errs = err_series.iloc[i-win:i].values
            if train_errs.size < win:
                continue
            nu, mu, sig = st.t.fit(train_errs)

            cur_date   = dates[i]
            slc        = panel_sorted[panel_sorted["release_date"] == cur_date]
            point_med  = slc["median_forecast"].iloc[0]   # centre at median + μ̂
            actual_val = slc["actual"].iloc[0]
            if np.isnan(actual_val):
                continue

            center_ts  = point_med + mu                   # overall centre
            for L in LEVELS:
                ts_hits[win][L] += in_band(center_ts, sig, nu, L, actual_val)
            ts_tot[win] += 1

        for L in LEVELS:
            rows.append({"Method": f"{panel_name}-TS-t_{win}m",
                         "Nominal": L,
                         "Empirical": ts_hits[win][L]/ts_tot[win] if ts_tot[win] else np.nan})

# coverage table
coverage_df = (pd.DataFrame(rows)
               .pivot(index="Method", columns="Nominal", values="Empirical")
               .sort_index())

print("\nEmpirical vs nominal coverage (XS-t centred at μ̂)")
print(coverage_df.to_string(float_format=lambda x: f"{x:0.3f}"))


COVID XS:   0%|          | 0/230 [00:00<?, ?it/s]

COVID TS 12m:   0%|          | 0/218 [00:00<?, ?it/s]

COVID TS 24m:   0%|          | 0/206 [00:00<?, ?it/s]

COVID TS 36m:   0%|          | 0/194 [00:00<?, ?it/s]

COVID TS 60m:   0%|          | 0/170 [00:00<?, ?it/s]

...

Full TS 24m:   0%|          | 0/242 [00:00<?, ?it/s]

Full TS 36m:   0%|          | 0/230 [00:00<?, ?it/s]

Full TS 60m:   0%|          | 0/206 [00:00<?, ?it/s]

Full TS 120m:   0%|          | 0/146 [00:00<?, ?it/s]


Empirical vs nominal coverage (XS-t centred at μ̂)
Nominal          0.500  0.600  0.700  0.800  0.900  0.950
Method                                                   
COVID-TS-t_120m  0.500  0.636  0.691  0.745  0.882  0.918
COVID-TS-t_12m   0.445  0.532  0.638  0.743  0.858  0.917
COVID-TS-t_24m   0.476  0.563  0.655  0.743  0.883  0.932
COVID-TS-t_36m   0.485  0.572  0.649  0.737  0.856  0.938
COVID-TS-t_60m   0.471  0.571  0.671  0.729  0.882  0.918
COVID-XS-t       0.222  0.274  0.343  0.417  0.483  0.587
Full-TS-t_120m   0.418  0.527  0.623  0.726  0.808  0.918
Full-TS-t_12m    0.445  0.531  0.642  0.748  0.858  0.917
Full-TS-t_24m    0.455  0.550  0.661  0.752  0.893  0.934
Full-TS-t_36m    0.452  0.557  0.639  0.765  0.874  0.943
Full-TS-t_60m    0.432  0.524  0.665  0.728  0.883  0.927
Full-XS-t        0.226  0.286  0.353  0.421  0.492  0.598
